# Spot market cleaning Script D354

In [1]:
# RUN EACH DAY AT 1PM
FORM = "D354"

# all_historical_data = True # take all historical submissions and clean it. 
# all_historical_data = False # take this week submissions and clean it. 

# create_dictionary = True
create_dictionary = False

# 0. Intro

## 0.0. Intro

In [2]:
import pyodbc
import pandas as pd
import numpy as np
import os
from pathlib import Path
from pandas.api.types import is_string_dtype
from pandas.api.types import is_numeric_dtype
import re
from datetime import datetime, timedelta
from tqdm import tqdm
from python_calamine import CalamineWorkbook # !pip install python-calamine
import shutil
from rapidfuzz import fuzz, utils # !pip install rapidfuzz
import xlwings as xw
from jarowinkler import *
import json 
import warnings
warnings.filterwarnings('ignore')
import time
import math

In [3]:
start_time = time.time()

### Create folder for today's report

In [4]:
path_CEC_Fernando = "P:\\ADMIN\\PIIRA\\CEC_Fernando"
path_CEC_Fernando_D354 = os.path.join(path_CEC_Fernando, FORM)
path_CEC_Fernando_D354

'P:\\ADMIN\\PIIRA\\CEC_Fernando\\D354'

In [5]:
wd = os.getcwd()
data_path = os.path.join(wd, "data")

# report_date = datetime.today().strftime('%Y%m%d')
now = datetime.now()

# report_date = str(now.year) + "{:02d}".format(now.month) + "{:02d}".format(now.day) + "_" +"H"+ "{:02d}".format(now.hour) +"M"+ "{:02d}".format(now.minute)
report_date = str(now.year) + "{:02d}".format(now.month) + "{:02d}".format(now.day) 

data_path = os.path.join(wd, "data", report_date)
Path(data_path).mkdir(parents=True, exist_ok=True)

path_CEC_Fernando_D354_data = os.path.join(path_CEC_Fernando_D354, "data", report_date)
path_CEC_Fernando_D354_data

'P:\\ADMIN\\PIIRA\\CEC_Fernando\\D354\\data\\20250117'

In [6]:
Path(path_CEC_Fernando_D354_data).mkdir(parents=True, exist_ok=True)

### Get location of Forms in P-Drive

In [7]:
def get_next_friday():
    today = datetime.today()
    days_to_friday = 4 - today.weekday()  # 4 represents Friday
    if days_to_friday <= 0:
        days_to_friday += 7
    next_friday = today + timedelta(days=days_to_friday)
    return next_friday

next_friday = get_next_friday()
next_friday

datetime.datetime(2025, 1, 24, 11, 32, 31, 39815)

In [8]:
# If today is Friday, Then define folder_next_friday as today.
# If today is not Friday, Then define as the actual next Friday.

# 4 represents Friday (0 is Monday, 1 is Tuesday, etc.)
if now.weekday()==4:
    folder_next_friday = 'Week_End_' + str(now.year)[2:] + str(now.month).zfill(2) + str(now.day).zfill(2)
    folder_year = str(now.year)
    folder_month = str(now.month).zfill(2) + "_" + now.strftime("%B")[:3]
    
else:
    folder_next_friday = 'Week_End_' + str(next_friday.year)[2:] + str(next_friday.month).zfill(2) + str(next_friday.day).zfill(2)
    folder_year = str(next_friday.year)
    folder_month = str(next_friday.month).zfill(2) + "_" + next_friday.strftime("%B")[:3]

# folder_month = '10_Oct'

path_spot_contract = "P:\\ADMIN\\PIIRA\\CEC_PIIRA Student Assistants\\Energy-PIIRA Inbox Email Collections\\"+folder_year+"\\"+folder_month+"\\D354_I Spot Contract\\"

path_archive = path_spot_contract + folder_next_friday  
path_archive

'P:\\ADMIN\\PIIRA\\CEC_PIIRA Student Assistants\\Energy-PIIRA Inbox Email Collections\\2025\\01_Jan\\D354_I Spot Contract\\Week_End_250117'

In [9]:
# path_archive_lastweek
def get_path_archive_lastweek(path_archive):
    # '241115'
    date_string = "20" + path_archive.split('\\')[-1].replace('Week_End_', '')
    date = datetime.strptime(date_string, '%Y%m%d').date() + timedelta(days=-7)

    folder_next_friday = 'Week_End_' + str(date.year)[2:] + str(date.month).zfill(2) + str(date.day).zfill(2)
    folder_year = str(date.year)
    folder_month = str(date.month).zfill(2) + "_" + date.strftime("%B")[:3]
    
    path_spot_contract = "P:\\ADMIN\\PIIRA\\CEC_PIIRA Student Assistants\\Energy-PIIRA Inbox Email Collections\\"+folder_year+"\\"+folder_month+"\\D354_I Spot Contract\\"
    path_archive = path_spot_contract + folder_next_friday  

    return path_archive

path_archive_lastweek = get_path_archive_lastweek(path_archive)
path_archive_lastweek

'P:\\ADMIN\\PIIRA\\CEC_PIIRA Student Assistants\\Energy-PIIRA Inbox Email Collections\\2025\\01_Jan\\D354_I Spot Contract\\Week_End_250110'

## 0.1. Dictionary

In [10]:
if create_dictionary:
    # workbook = CalamineWorkbook.from_path(os.path.join(path_CEC_Fernando_D354, "dictionary.xlsx"))
    workbook = CalamineWorkbook.from_path("dictionary.xlsx")
    
    vars = workbook.sheet_names
    vars.remove("KEYS")
    print(vars)
    
    df_Dictionary = pd.DataFrame()
    dict_name = {}
    for var in vars:
        # print(var)
        form_list = workbook.get_sheet_by_name(var).to_python()
        
        if len(form_list)>0:
            df_ = pd.DataFrame(data=form_list[1:], columns=form_list[0])
            dict_name[var] = form_list[0]
            for col in df_.columns:
                df_Dictionary[col] = df_[col].astype(str)
                
        # if len(form_list)==0:
        #     vars.remove(var)
    

In [11]:
if create_dictionary:
    dict_Directory = {}
    for key, value in dict_name.items():
        # print(key + " -- " + str(value))
        dict_ = {}
        for x in df_Dictionary[value[0]].unique():
            # print(x)
            list_values = list(df_Dictionary[df_Dictionary[value[0]]==x][value[1]].unique())
            list_values = [x.replace('n/a', 'nan') for x in list_values]
            dict_[x] = list_values
        # del dict_['']    
        dict_Directory[key] = dict_
    
    for key, value in dict_Directory.items():
        # print(key)
        # print(dict_Directory[key].keys())
        if 'nan' in list(dict_Directory[key].keys()): del dict_Directory[key]['nan']
    
    with open("dict_Directory.json", "w") as outfile: 
        json.dump(dict_Directory, outfile)

In [12]:
if not create_dictionary: 

    with open("dict_Directory.json","r") as file:
        dict_Directory = json.load(file)
    
    for k, v in dict_Directory.items():
        if 'NaN' in list(dict_Directory[k].keys()): del dict_Directory[k]['NaN']

In [13]:
list(dict_Directory.keys())

['Company',
 'Position Seq. Num.',
 'CPID',
 'Counterparty Contact',
 'TL',
 'Sell_Buy',
 'Product Type',
 'Product',
 'Delivery Method',
 'Delivery Location',
 'Price Basis Method',
 'Reference Product']

## 0.2. Submitted

In [14]:
list_order_col = [
'Report Period',
'Company Name',
'Company ID Number',
'Date Filed',
'Date of Contract',
'Contract ID Number',
'Position Sequence Number',
'CPID',
'Trading Counterparty Entity',
'Trading Counterparty Entity Contact Name',
'Spot Market Trading Location',
'Seller or Buyer',
'Product Type',
'Product Name',
'Contract Volume',
'Contract Delivery Month',
'Delivery Method',
'Marine Vessel Name',
'Marine Vessel IMO Number',
'Delivery Location',
'Pipeline Delivery Sub-cycle',
'Delivery Start Date',
'Delivery End Date',
'Price Basis Method',
'Reference Product',
'Reference Month',
'Price Differential vs. Reference',
"Pricing Event Type",
"Pricing Event Date",
'Pricing Start Date',
'Pricing End Date',
'NYMEX Price',
'Cash Price',
'Broker Used Entity',
'Broker Used Entity Contact Name',
'Reported to OPIS',
"File form",
]

In [15]:
def similar_colname(string):

    def capitalize_words(text):
        return " ".join(word.capitalize() for word in text.split())
    
    dict_ = {}
    for col in list_order_col: dict_[col] = round(jaro_similarity(capitalize_words(string), col),2)*100
    
    if max(dict_.values())>=90: 
        new_string = max(dict_, key= lambda x: dict_[x])
        return new_string
    else: return string

In [16]:
def get_trading_form(filepath):
    workbook = CalamineWorkbook.from_path(filepath)


    form_list = workbook.get_sheet_by_name(workbook.sheet_names[0]).to_python()

    Report_Period = form_list[4][1]
    Company_Name = form_list[7][1]
    Company_ID_Number = form_list[8][1]
    Company_Address = form_list[9][1]
    Company_Email = form_list[10][1]
    Company_Phone_Number = form_list[11][1]
    Contact_Name = form_list[12][1]
    Contact_Phone_Number = form_list[13][1]
    Contact_Email = form_list[14][1]
    Date_Filed = form_list[15][1]

    form_list = workbook.get_sheet_by_name(workbook.sheet_names[1]).to_python()
    
    df = pd.DataFrame(data=form_list[1:], columns=form_list[0])
    if len([col for col in df.columns if col=='']) >= 2: df = pd.DataFrame(data=form_list[2:], columns=form_list[1])


    if 'Was This Transaction Reported to OPIS by: You, Counterparty, your Broker, Other Party (specify) or None' in df.columns:
        df.columns = df.columns.map(lambda col: 'Was This Transaction Reported to OPIS' if col.lower().find('Was This Transaction Reported to OPIS'.lower())>-1 else col)
    
    df.columns = [string.split("(")[0] for string in df.columns]
    df.columns = [string.split('\r')[0] for string in df.columns]
    df.columns = [string.strip() for string in df.columns]

    df = df[np.logical_or(df['Trading Counterparty Entity']!='', df['Contract ID Number']!='')]

    df.columns = [similar_colname(string) for string in df.columns]

    df["Report Period"] = Report_Period
    df["Company Name"] = Company_Name
    df["Company ID Number"] = Company_ID_Number
    # df["Company Address"] = Company_Address
    # df["Company Email"] = Company_Email
    # df["Company Phone Number"] = Company_Phone_Number
    # df["Contact Name"] = Contact_Name
    # df["Contact Phone_Number"] = Contact_Phone_Number
    # df["Contact Email"] = Contact_Email
    df["Date Filed"] = Date_Filed
    df["File form"] = filepath 

    if 'Was This Transaction Reported to OPIS' in df.columns:
        df.rename(columns={'Was This Transaction Reported to OPIS': 'Reported to OPIS'}, inplace=True)
    if 'Trading Counterparty Entity Name' in df.columns:
        df.rename(columns={'Trading Counterparty Entity Name': 'Trading Counterparty Entity'}, inplace=True)
    
    df = df.reset_index(drop=True)

    if len(df.columns) < len(list_order_col):
        for col in list(set(list_order_col) - set(df.columns)):
            df[col] = None
            
    df = df[list_order_col]

    return df

In [17]:
def get_files_with_timestamps(directory):
    files = {}
    for filename in os.listdir(directory):
        filepath = os.path.join(directory, filename)
        if os.path.isfile(filepath):
            stat = os.stat(filepath)
            files[filename] = {}
            files[filename]['date'] = datetime.fromtimestamp(stat.st_mtime)
            files[filename]['path'] = directory
    return files

In [18]:
# if all_historical_data: path_piira = 'P:\\FTD\\FOSSIL FUELS\\PIIRA Reporting\\Archive\\2024\\D354_I Spot Contract\\Final Reports'
path_piira = os.path.join(path_archive, "Trade")
path_piira

'P:\\ADMIN\\PIIRA\\CEC_PIIRA Student Assistants\\Energy-PIIRA Inbox Email Collections\\2025\\01_Jan\\D354_I Spot Contract\\Week_End_250117\\Trade'

In [19]:
path_piira_lastweek = os.path.join(path_archive_lastweek, "Trade")
path_piira_lastweek

'P:\\ADMIN\\PIIRA\\CEC_PIIRA Student Assistants\\Energy-PIIRA Inbox Email Collections\\2025\\01_Jan\\D354_I Spot Contract\\Week_End_250110\\Trade'

In [20]:
files_with_timestamps = get_files_with_timestamps(path_piira) 
if os.path.exists(path_piira_lastweek):
    files_with_timestamps_lastweek = get_files_with_timestamps(path_piira_lastweek) 
    files_with_timestamps.update(files_with_timestamps_lastweek)
len(files_with_timestamps.keys())

122

In [21]:
list_key = list(files_with_timestamps.keys())
for key in list_key:
    if key.find('~$') > -1:
        del files_with_timestamps[key]

In [22]:
dict_files_timestamps = {}
for key in files_with_timestamps.keys():
    dict_files_timestamps[key] = files_with_timestamps[key]['date']
dict_files_timestamps

{'D354_I_240723_HFSinclair_Trade_ID1013332.xlsx': datetime.datetime(2025, 1, 17, 8, 3, 46, 250496),
 'D354_I_250101_Boyett_Trade_ID1013297.xlsx': datetime.datetime(2025, 1, 15, 15, 40, 50, 311652),
 'D354_I_250106_Mieco_Trade_ID1013272.xlsx': datetime.datetime(2025, 1, 15, 7, 24, 46, 543492),
 'D354_I_250106_Sage_Trade_ID1013302.xlsx': datetime.datetime(2025, 1, 16, 6, 53, 14, 520736),
 'D354_I_250107_Sage_Trade_ID1013302.xlsx': datetime.datetime(2025, 1, 16, 6, 53, 18, 443018),
 'D354_I_250108_Sage_Trade_ID1013302.xlsx': datetime.datetime(2025, 1, 16, 6, 53, 23, 916399),
 'D354_I_250109_Sage_Trade_ID1013302.xlsx': datetime.datetime(2025, 1, 16, 6, 53, 29, 990060),
 'D354_I_250109_TACenergy_Trade_ID1013160.xlsx': datetime.datetime(2025, 1, 13, 10, 18, 43, 238060),
 'D354_I_250110_0916161800_BP_Trade_ID1013142.xlsx': datetime.datetime(2025, 1, 13, 7, 3, 50, 472117),
 'D354_I_250110_200003900_PBF_Trade_ID1013145.xlsx': datetime.datetime(2025, 1, 13, 7, 3, 36, 524713),
 'D354_I_250110_255

In [23]:
sorted_items = sorted(dict_files_timestamps.items(), key=lambda item: item[1])
forms = [item[0] for item in sorted_items]
len(forms)

122

In [24]:
# forms = [f for f in os.listdir(path_piira) if f.find(".xls")>0]
# forms = [form for form in forms if form.find("Trade")>-1]
# forms = [form for form in forms if form.find("SageRefined")==-1]
# forms = [form for form in forms if form.find("Mieco")==-1]
# len(forms) MIECO

In [25]:
def get_forms_errors(forms):
    forms_errors = [form for form in forms if form.find("Trade")==-1]
    forms_errors = forms_errors + [form for form in forms if form.find("SageRefined")>-1]
    forms_errors = forms_errors + [form for form in forms if form.find("Mieco")>-1]   
    forms_errors = forms_errors + [form for form in forms if form.find("MIECO")>-1]

    forms_forms = [form for form in forms if form.find("Trade")>-1]
    forms_forms = [form for form in forms_forms if form.find("SageRefined")==-1]
    forms_forms = [form for form in forms_forms if form.find("Mieco")==-1]   
    forms_forms = [form for form in forms_forms if form.find("MIECO")==-1] 

    return forms_forms, forms_errors

In [26]:
forms, errors = get_forms_errors(forms)

In [27]:
filepath = os.path.join(files_with_timestamps[forms[0]]['path'], forms[0])
df = get_trading_form(filepath)

# errors = []
for f in forms[1:]:
    print(f)
    try:
        # filepath = os.path.join(path_piira, f)
        filepath = os.path.join(files_with_timestamps[f]['path'], f)
        df_ = get_trading_form(filepath) 
        df = pd.concat([df, df_], axis='rows')    
    except:
        errors.append(f)

D354_I_250103_Tartan_Trade_ID1012972.xlsx
D354_I_250103_255000101_P66_Trade_ID1012973.xlsx
D354_I_250103_Murex_Trade_ID1012974.xlsx
D354_I_250106_058050800_Shell_Trade_ID1012980.xlsx
D354_I_250103_98181445_BB_Trade_ID1012982.xlsx
D354_I_250103_100100_Valero_Trade_ID1012984.xlsx
D354_I_250103_200003900_PBF_Trade_ID1012989.xlsx
D354_I_250106_Idemitsu_Trade_ID1012995.xlsx
D354_I_250102_NorthAmerican_Trade_ID1013008.xlsx
D354_I_250106_255000101_P66_Trade_ID1013011.xlsx
D354_I_250106_BP_Trade_ID1013015.xlsx
D354_I_250106_PBFenergy_Trade_ID1013016.xlsx
D354_I_250106_Murex_Trade_ID1013019.xlsx
D354_I_250106_Tartan_Trade_ID1013020.xlsx
D354_I_250106_Chevron_Trade_ID1013021.xlsx
D354_I_250107_Valero_Trade_ID1013017.xlsx
D354_I_250107_058050800_Shell_Trade_ID1013040.xlsx
D354_I_250106_Marathon_Trade_ID1013045.xlsx
D354_I_250106_PetroDiamond_Trade_ID1013057.xlsx
D354_I_250107_0916161800_BP_Trade_ID1013060.xlsx
D354_I_250107_Marathon_Trade_ID1013059.xlsx
D354_I_250107200003900_PBF_Trade_ID1013061.

In [28]:
errors

['D354_I_250106_Mieco_Trade_ID1013272.xlsx',
 'D354_I_250106_MIECO_Trade_ID1013038.xlsx',
 'D354_I_250106_MIECO_Trade_ID1013038 (2).xlsx',
 'D354_I_250113_MIECO_Trade_ID1013300.xlsx',
 'D354_I_250103_Murex_Trade_ID1012974.xlsx',
 'D354_I_250106_Murex_Trade_ID1013019.xlsx',
 'D354_I_250107_MUREX_Trade_ID1013063.xlsx',
 'D354_I_250109_Valero_Trade_ID1013124.xlsx',
 'D354_I_250115_Murex_Trade_ID1013310.xlsx',
 'D354_I_250116_MUREX_Trade_ID1013335.xlsx']

In [29]:
# add a new column with the timestamps
df['filename'] = df['File form'].map(lambda string: string.split('\\')[-1])

# add new column with vaues from dict_files_timestamps 
df['timestamps'] = df['filename'].map(dict_files_timestamps)

# delete filename column
df.drop('filename', axis=1, inplace=True)

# put the column timestamps at the beginning
df = df[['timestamps'] + [col for col in df.columns if col != 'timestamps']]

# give format to column timestamps
df['timestamps'] = df['timestamps'].map(lambda x: x.strftime("%Y-%m-%d %H:%M:%S"))

In [30]:
# Sunit is going to work on a logic for identifying duplicates
# Identify duplicated
# 7 columns

In [31]:
filename = "Trading_Submitted.csv"

df.to_csv(filename)
df.to_csv(os.path.join(data_path, filename))

# df.to_csv(os.path.join(path_CEC_Fernando_D354, filename))
df.to_csv(os.path.join(path_CEC_Fernando_D354_data, filename))

In [32]:
df_Submitted = df.copy()

In [33]:
df_files = pd.DataFrame(data=forms, columns=['File'])
# df_files['path_piira'] = path_piira
df_files['path_piira'] = df_files["File"].map(lambda string: files_with_timestamps[string]['path'])
df_files['Added'] = np.where(~df_files['File'].isin(errors), df_files['File'], None)
df_files['Error'] = np.where(df_files['File'].isin(errors), df_files['File'], None)
df_files['N'] = df_files.index
df_files['N'] = df_files['N'] + 1
df_files = df_files[['N', 'File', 'Added', 'Error', 'path_piira']]

In [34]:
# df_Submitted['File'] = df_Submitted["File form"].map(lambda string: string.split('\\')[-1])
# df_S = df_Submitted[['File', 'Company Name']].copy()

In [35]:
filename = "Trading_Files.csv"

df_files.to_csv(filename)
df_files.to_csv(os.path.join(data_path, filename))

# df_files.to_csv(os.path.join(path_CEC_Fernando_D354, filename))
df_files.to_csv(os.path.join(path_CEC_Fernando_D354_data, filename))

In [36]:
# if not all_historical_data: filename = "Trading_errors.txt"
# if all_historical_data: filename = "Trading_errors_all.txt"

In [37]:
# with open(filename, "w", encoding="utf-8") as output:
#     for item in errors:
#         output.write(str(item) + "\n")

# with open(os.path.join(path_CEC_Fernando_D354, filename), "w", encoding="utf-8") as output:
#     for item in errors:
#         output.write(str(item) + "\n")

In [38]:
# with open(os.path.join(data_path, filename), "w", encoding="utf-8") as output:
#     for item in errors:
#         output.write(str(item) + "\n")

# with open(os.path.join(path_CEC_Fernando_D354_data, filename), "w", encoding="utf-8") as output:
#     for item in errors:
#         output.write(str(item) + "\n")

# 1. Cleaning

## 1.1. Trading

### Intro

In [39]:
def str_to_date(string):

    string = string.strip()
    string = string.replace("  ", " ")
    string = string.replace("   ", " ")

    if string=='nan': return pd.NaT
    if string=='': return pd.NaT
    if string=='(Empty)': return pd.NaT
    if string=='NaT': return pd.NaT
    
    regex = '^\d{1,2}(/|//)\d{1,2}(/|//)\d{4}$'
    match = re.search(regex, string)
    if match: 
        string_ = string.replace("//", "/")
        month, day, year = string_.split("/")
        # print(regex)
        
    regex = '^\d{4}(-)\d{1,2}(-)\d{1,2}$'
    match = re.search(regex, string)
    if match: 
        year, month, day = string.split("-")
        # print(regex)

    regex = '^\d{4}(-)\d{1,2}(-)\d{1,2}( )\d{1,2}(:)\d{1,2}(:)\d{1,2}$'
    match = re.search(regex, string)
    if match: 
        year, month, day = string.split(" ")[0].split("-")
        # print(regex)
        
    # string = '10.17.24'
    # string = '10.17.2024'
    regex = '^\d{1,2}(\.)\d{1,2}(\.)\d{2,4}$'
    match = re.search(regex, string)
    if match: 
        month, day, year = string.split(".")
        if len(year)==2: year = '20' + year
        # print(regex)
    
    regex = '^\d{1,2}(-)\d{1,2}(-)\d{4}$'
    match = re.search(regex, string)
    if match: 
        month, day, year = string.split("-")
        # print(regex)
        
    # string = "10/4/24 - 10/6/24"
    regex = '^\d{1,2}(/)\d{1,2}(/)\d{2}( - )\d{1,2}(/)\d{1,2}(/)\d{2}$'
    match = re.search(regex, string)
    if match: 
        month, day, year = string.split(" - ")[1].split("/")
        year = "20" + year
        # print(regex)
    
    # string = "10/4-6/2024"
    regex = '^\d{1,2}(/)\d{1,2}(-)\d{1,2}(/)\d{4}$'
    match = re.search(regex, string)
    if match: 
        month = string.split("/")[0]
        year = string.split("/")[2]
        day = string.split("-")[1].split("/")[0]
        # print(regex)
        
    # string = "10-04-2024 - 10-06-2024"
    regex = '^\d{1,2}(-)\d{1,2}(-)\d{4}( - )\d{1,2}(-)\d{1,2}(-)\d{4}$'
    match = re.search(regex, string)
    if match: 
        month, day, year = string.split(" - ")[1].split("-")
        # print(regex)
        
    # string = '3/22-3/24/2024'
    # string = "10/4 - 10/6/2024"
    regex = '^\d{1,2}(/)\d{1,2}(-| - )\d{1,2}(/)\d{1,2}(/)\d{4}$'
    match = re.search(regex, string)
    if match:
        string_ = string.replace(" - ", "-")
        month, day, year = string_.split("-")[1].split("/")
        # print(regex)
    
    # string = "9/2024 & 10/2024"
    regex = '^\d{1,2}(/)\d{4}( & )\d{1,2}(/)\d{4}$'
    match = re.search(regex, string)
    if match: 
        month, year = string.split(" & ")[1].split("/")
        day = "0"
        # print(regex)

    # string = "10/4/2024 - 10/6/2024"
    # string = '10/11/24 - 10/13/24 - Revised'
    regex = '^\d{1,2}(/)\d{1,2}(/)\d{2,4}( - )\d{1,2}(/)\d{1,2}(/)\d{2,4}'
    match = re.search(regex, string)
    if match: 
        month, day, year = string.split(" - ")[1].split("/")
        if len(year)==2: year = '20' + year
        # print(regex)
        
    # string = "2024-10-07 06:45:03.306000"
    regex = '^\d{4}(-)\d{1,2}(-)\d{1,2}( )\d{1,2}(:)\d{1,2}(:)\d{1,2}(\.)\d{6}$'
    match = re.search(regex, string)
    if match: 
        year, month, day = string.split(" ")[0].split("-")
        # print(regex)
        
    # string = '3/1/2024-3/3/2024'
    # string = '3/1/2024 to 3/3/2024'
    # string = '3/1/2024 - 3/3/2024'
    # string = '7.2.2024-7.3.2024'
    # string = '8/2/2024- 8/4/2024'
    regex = '^\d{1,2}(/|\.)\d{1,2}(/|\.)\d{4}( to |-| - |- )\d{1,2}(/|\.)\d{1,2}(/|\.)\d{4}$'
    match = re.search(regex, string)
    if match: 
        string_ = string.replace(" ","").replace("to","-").replace(".","/")
        month, day, year = string_.split("-")[1].split("/")

    # string = '010/14/2024'
    regex = '^\d{3}(/)\d{1,2}(/)\d{4}$'
    match = re.search(regex, string)
    if match: 
        month, day, year = string.split("/")

    # string = '05-26-2024 - 05-27-2024'
    regex = '^\d{1,2}(-)\d{1,2}(-)\d{4}(-| - )\d{1,2}(-)\d{1,2}(-)\d{4}$'
    match = re.search(regex, string)
    if match: 
        string_ = string.replace(" - ","-")
        _, _, _, month, day, year = string_.split("-")

    # if year=='1900' and month=='1' and day=='1': 
    #     return pd.NaT

    try:
        # convert to int then to str: 01 ------> 1 also 016 --------> 16
        year = str(int(year))
        month = str(int(month))
        day = str(int(day))
        
        date_object = datetime.strptime(year +"-"+ month +"-"+ day, '%Y-%m-%d').date()
        return date_object
    except UnboundLocalError:
        return pd.NaT
    except IndexError:
        return pd.NaT
    except ValueError:
        return pd.NaT
    # except Exception as e:
    # except (ValueError, UnboundLocalError, IndexError) as e:

In [40]:
# ValueError: time data 
string = '2024-10-0'
str_to_date(string)

NaT

In [41]:
def month_first_letters_to_number(string):
    months_dict = {'Jan': 1, 'Feb': 2, 'Mar': 3, 'Apr': 4, 'May': 5, 'Jun': 6, 'Jul': 7, 'Aug': 8, 'Sep': 9, 'Oct': 10, 'Nov': 11, 'Dec': 12}
    
    if string in months_dict.keys(): return months_dict[string] 
    else: return None 

In [42]:
def month_fullname_to_number(string):
    months_dict = {'January': 1, 
                   'February': 2, 
                   'March': 3, 
                   'April': 4, 
                   'May': 5, 
                   'June': 6, 
                   'July': 7, 
                   'August': 8, 
                   'September': 9, 
                   'October': 10, 
                   'November': 11, 
                   'December': 12}
    
    if string in months_dict.keys(): return months_dict[string] 
    else: return None 

In [43]:
def find_full_month_name_to_number(string):
    list_month_fullname = ['January', 'February', 'March', 'April','May','June','July','August','September','October','November', 'December']
    for month in list_month_fullname:
        # print(str(string.find(month)))
        if string.find(month)>-1: return month_fullname_to_number(month)    
    return None

In [44]:
def str_to_month(string):

    if string is None: return None
        
    string = string.strip()
    string = string.replace("  ", " ")
    string = string.replace("   ", " ")

    if string=='': return None
    if string=='-': return None
    if string=='(Empty)': return None
    if string=='(blank)': return None
    if string=='NOT APPLICABLE': return None
    if string=='#Err': return None
        
    # string = 'Nov'
    regex = '^[a-zA-Z]{3}$'
    match = re.search(regex, string)
    if match: 
        month = str(month_first_letters_to_number(string[0].upper() + string[1:].lower()))
        year = str(now.year)
        day = str(now.day)
    
    # string = "2024-10-01"
    regex = '^\d{4}(-)\d{1,2}(-)\d{1,2}$'
    match = re.search(regex, string)
    if match: 
        year, month, day = string.split("-")

    # string = '2024-10-17 05:00:00'
    regex = '^\d{4}(-)\d{1,2}(-)\d{1,2}( )\d{1,2}(:)\d{1,2}(:)\d{1,2}$'
    match = re.search(regex, string)
    if match: 
        year, month, day = string.split(" ")[0].split("-")

    # string = '10/09/2024'
    regex = '^\d{1,2}(/)\d{1,2}(/)\d{4}$'
    match = re.search(regex, string)
    if match: 
        month, day, year = string.split("/")

    # string = 'Nov 2024'
    regex = '^[a-zA-Z]{3}( )\d{4}$'
    match = re.search(regex, string)
    if match: 
        month, year = string.split(" ")
        month = str(month_first_letters_to_number(month))
        day = str(now.day)
        
    # string = 'November-2024'
    if find_full_month_name_to_number(string) is not None and string.find(str(now.year))>-1: 
        month = str(find_full_month_name_to_number(string))
        year = str(now.year)
        day = str(now.day)

    # string = 'NOV-24'
    regex = '^[a-zA-Z]{3}(-)\d{2}$'
    match = re.search(regex, string)
    if match and month_first_letters_to_number(string.split("-")[0][0].upper() + string.split("-")[0][1:].lower()) is not None: 
        month = str(month_first_letters_to_number(string.split("-")[0][0].upper() + string.split("-")[0][1:].lower()))
        year = '20' + string.split("-")[1]
        day = str(now.day)

    # string = 'November'
    if string in ['January', 'February', 'March', 'April','May','June','July','August','September','October','November', 'December']:
        month = str(month_fullname_to_number(string))
        year = str(now.year) 
        day = str(now.day)

    # string = 'AUGUST'
    string_ = string[0].upper() + string[1:].lower()
    if string_ in ['January', 'February', 'March', 'April','May','June','July','August','September','October','November', 'December']:
        month = str(month_fullname_to_number(string_))
        year = str(now.year) 
        day = str(now.day)

    # string = 'Dec-2024'
    regex = '^[a-zA-Z]{3}(-)\d{4}$'
    match = re.search(regex, string)
    if match and month_first_letters_to_number(string.split("-")[0][0].upper() + string.split("-")[0][1:].lower()) is not None: 
        month = str(month_first_letters_to_number(string.split("-")[0][0].upper() + string.split("-")[0][1:].lower()))
        year = string.split("-")[1]
        day = str(now.day)  

    # string = 'Sept'
    regex = '^[a-zA-Z]{4}$'
    match = re.search(regex, string)
    if match and type(month_first_letters_to_number(string[0].upper() + string[1:3].lower()))==int: 
        month = str(month_first_letters_to_number(string[0].upper() + string[1:3].lower()))
        year = str(now.year)
        day = str(now.day)
  
    try:
        date_object = datetime.strptime(year +"-"+ month +"-"+ day, '%Y-%m-%d').date().strftime("%b-%Y")
        return date_object
    except UnboundLocalError:
        return None
    except ValueError:
        return None

In [45]:
def str_to_float(string):
    try:
        return float(string)
    except ValueError as e:
        return np.nan
    except TypeError as e:
        return np.nan

In [46]:
def match_dictio(string, dictio, case_sensitive=True):

    try:
        string = string.strip()
    except AttributeError:
        return None
    
    if string=='': return None
    if string=='-': return None
    if string=='(Empty)': return None
    if string=='(blank)': return None
    if string=='NOT APPLICABLE': return None
    if string=='#Err': return None

    if case_sensitive:
        for key, values in dictio.items():
            if string==key: return key
            if string in values: return key
    
    if not case_sensitive: 
        string = string.lower()
        dictio_ = {}
        for key, value in dictio.items():
            dictio_[str(key).lower()] = [str(v).lower() for v in value]  
        dict_key_to_Key = {str(k).lower(): str(k) for k in dictio.keys()}
        for key, values in dictio_.items():
            if string==key: return dict_key_to_Key[key]
            if string in values: return dict_key_to_Key[key]
    return None


In [47]:
def clean_string(string):
    if string=='': return None
    if string=='-': return None
    if string=='(Empty)': return None
    if string=='(blank)': return None
    if string=='NOT APPLICABLE': return None
    if string=='#Err': return None
    if string=='None': return None
    if string=='nan': return None
    if string=='N/A': return None
    if string=='n/a': return None
    if string=='NA': return None
    if string=='na': return None
    if string==' ': return None
    if string=='  ': return None
    if string=='none': return None
    return string

In [48]:
def extract_numbers(string):
    if string=='nan': return np.nan
    if string=='': return np.nan

    try:
        return re.findall(r"[-+]?\d*\.\d+|\d+", string)[0]
    except TypeError:
        return None
    except IndexError:
        return None

### Variables

#### Report_Period

In [49]:
df['Report Period'] = df["Report Period"].astype(str)

In [50]:
df['Report Period'].unique()

array(['2025-01-03', '1/3 - 1/5/2025', '2025-01-06',
       '1/3/2025 - 1/5/2025', '2025-01-02', '2025-01-07', '01.06.2025',
       '01.07.2025', '01/07/2025', '2025-01-08', '01.08.2025',
       '01-08-2025', '2025-01-09', '01.09.2025', '2025-01-10',
       '01-09-2025', '01/09/2025', '1/10/2025 - 1/12/2025', '01.10.2025',
       '1/10-12/2025', '01-10-2025 - 01-12-2025', '1/10 - 1-12/2025',
       '2025-01-13', '01.13.2025', '01-13-2025', '01/13/2025',
       '2025-01-14', '01-14-2025', '01/14/2025', '2025-01-15',
       '2025-01-01', '01.14.2025 - 01.15.2025', '01-15-2025',
       '2025-01-16', '2024-07-23', '01.16.2025', '2025-01-17'],
      dtype=object)

In [51]:
df['Report Period'] = df["Report Period"].map(lambda x: str_to_date(x))

In [52]:
df['Report Period'].unique()

array([datetime.date(2025, 1, 3), datetime.date(2025, 1, 5),
       datetime.date(2025, 1, 6), datetime.date(2025, 1, 2),
       datetime.date(2025, 1, 7), datetime.date(2025, 1, 8),
       datetime.date(2025, 1, 9), datetime.date(2025, 1, 10),
       datetime.date(2025, 1, 12), NaT, datetime.date(2025, 1, 13),
       datetime.date(2025, 1, 14), datetime.date(2025, 1, 15),
       datetime.date(2025, 1, 1), datetime.date(2025, 1, 16),
       datetime.date(2024, 7, 23), datetime.date(2025, 1, 17)],
      dtype=object)

#### Company_Name

In [53]:
df['Company Name'].unique()

array(['bp Products North America Inc.', 'Tartan Oil LLC',
       'Phillips 66 Company', 'Shell Trading US Company',
       'BB Energy USA LLC',
       'Ultramar, Inc. / Valero Refining Co. - California', 'PBF Energy',
       'Idemitsu Apollo Corporation', 'North American Fuel Corporation',
       'Chevron Products Company, a division of Chevron U.S.A. Inc.',
       'Marathon Petroleum', 'Petro-Diamond Incorporated ',
       'VITOL AVIATION COMPANY', 'Sage Refined Products, Ltd.',
       'QuikTrip Corporation', 'Hartland Fuels LLC DBA HTP Energy',
       'SBR Marketing', 'TACenergy, LLC', 'STAN BOYETT & SON, INC',
       'HF SINCLAIR CORPORATION', 'NGL CRUDE LOGISTICS, LLC'],
      dtype=object)

In [54]:
dict_Directory['Company']

{'US Venture': ['  U.S. Venture, Inc',
  'U.S Venture',
  'US Venture',
  'US VENTURE INC',
  'US VENTURE INC-PRD'],
 'Alaska Airlines': ['Alaska Airlines, Inc'],
 'Allegiant Air': ['Allegiant Air, LLC'],
 'Altair': ['Altair Paramount, LLC'],
 'Ampol': ['Ampol US Trading LLC', 'AMPOL US TRADING LLC-PRD', 'Ampol USA'],
 'Aramco': ['Aramco Trading ',
  'Aramco Trading Americas LLC',
  'ARAMCO TRADING AMERICAS LLC-PRD'],
 'Archer-Daniels': ['Archer-Daniels-Midland Company'],
 'Avfuel': ['Avfuel Corporation', 'Avfuel Corporation '],
 'BB': ['BB Energy',
  'BB Energy USA LLC',
  'BB ENERGY USA LLC-PRD',
  'BB Energy USA LLC-SDP'],
 'Biosphere': ['Biosphere Fuels, LLC'],
 'Boyett': ['BOYETT', 'Boyett Petroleum', 'STAN BOYETT & SON, INC'],
 'BP': ['BP',
  'BP NA Petroleum',
  'BP North America',
  'BP North America Pet',
  'BP North America Petroleum',
  'BP NORTH AMERICA PETROLEUM INC.',
  'BP NORTH AMERICA PETROLEUM, A DIVISION OF BP PRODUCTS NORTH AMERICA INC.',
  'BP NORTH AMERICA PETROLE

In [55]:
for key, values in dict_Directory['Company'].items():
    df['Company Name'] = df['Company Name'].replace(values, key)

In [56]:
set(df['Company Name'].unique()) <= set(dict_Directory['Company'].keys())

False

In [57]:
# Nombres de company que no estan en keys
nokey_list = list(set(df['Company Name'].unique()) - set(dict_Directory['Company'].keys()))
nokey_list = [str(nokey) for nokey in nokey_list]

if "nan" in nokey_list:
    nokey_list.remove("nan")

nokey_list

['Hartland Fuels LLC DBA HTP Energy',
 'Sage Refined Products, Ltd.',
 'NGL CRUDE LOGISTICS, LLC',
 'Marathon Petroleum']

In [58]:
key_list = list(dict_Directory['Company'].keys())
key_list

['US Venture',
 'Alaska Airlines',
 'Allegiant Air',
 'Altair',
 'Ampol',
 'Aramco',
 'Archer-Daniels',
 'Avfuel',
 'BB',
 'Biosphere',
 'Boyett',
 'BP',
 'Castleton',
 'CCMT',
 'Chase Inc',
 'Chevron',
 'Clean2050',
 'CMA CGM',
 'Cockett',
 'Crimson',
 'DAN Bunkering',
 'Diamond Green',
 'Eastern Aviation',
 'Elbow River',
 'ENEOS',
 'Enpramex',
 'Epsilon',
 'Equilon',
 'Flyers',
 'FSM',
 'General Pet.',
 'Glander Int.',
 'Glencore',
 'GreenAmerica',
 'Gresham',
 'Gunvor',
 'Hartland',
 'Herbst',
 'HF Sinclair',
 'HTP',
 'Idemitsu',
 'JACO',
 'Jaco Oil',
 'Kern',
 'Keyera',
 'Kinder Morgan',
 'KPI',
 'Lincoln Oil',
 'Mansfield',
 'Marathon',
 'Martinez Renewables',
 'Maverik',
 'McCall',
 'Mieco',
 'Monjasa',
 'Murex',
 'Musket',
 'n/a',
 'NAFCO',
 'Neste',
 'NGL',
 'N.A.Fuels',
 'Offen',
 'P66',
 'Parkland',
 'PBF',
 'Petro Diamond',
 'Petro Star',
 'Petrochina',
 'Petrocom',
 'Pilot Travel',
 'QuickTrip',
 'QT Fuels',
 'Rebel Oil',
 'REG Mktg.',
 'RIL USA',
 'River City',
 'Rubis',


In [59]:
dictio_ = {}
for nokey in nokey_list:
    print(nokey)
    dict_ = {}
    for key in key_list:
        value_list = list(dict_Directory['Company'][key])
        dict_[key] = np.max(np.array([round(fuzz.token_set_ratio(value.upper(), nokey.upper()),2) for value in value_list]))
            
    key_max = max(dict_, key= lambda x: dict_[x])
    print(key_max)
    dictio_[nokey] = key_max

dictio_

Hartland Fuels LLC DBA HTP Energy
HTP
Sage Refined Products, Ltd.
Shell
NGL CRUDE LOGISTICS, LLC
NGL
Marathon Petroleum
Marathon


{'Hartland Fuels LLC DBA HTP Energy': 'HTP',
 'Sage Refined Products, Ltd.': 'Shell',
 'NGL CRUDE LOGISTICS, LLC': 'NGL',
 'Marathon Petroleum': 'Marathon'}

In [60]:
df["Company Name"] = df["Company Name"].replace(dictio_)

# Nombres de company que no estan en keys
nokey_list = list(set(df["Company Name"].unique()) - set(dict_Directory['Company'].keys()))
nokey_list

[]

#### Company_ID_Number

In [61]:
df['Company ID Number'].unique()

array(['BPN-0916161800', '', 255000101.0, '76-058050800', 98181445.0,
       100100.0, 200003900.0, 'Unknown', 399541646.0, 'XXXXXXXXXXXX',
       710891110.0], dtype=object)

In [62]:
# dict_Directory['Company']
dict_Directory.keys()

dict_keys(['Company', 'Position Seq. Num.', 'CPID', 'Counterparty Contact', 'TL', 'Sell_Buy', 'Product Type', 'Product', 'Delivery Method', 'Delivery Location', 'Price Basis Method', 'Reference Product'])

#### Date_Filed

In [63]:
df['Date Filed'] = df['Date Filed'].astype(str)

In [64]:
df['Date Filed'].unique()

array(['2025-01-06', '2025-01-06 08:44:46.980000', '2025-01-02',
       '2025-01-07 08:07:59.190000', '2025-01-07', '', '2025-01-08',
       '2025-01-08 08:25:42.430000', '01/08/2025', '2025-01-04',
       '2025-01-09 08:42:16.610000', '2025-01-09', '2025-01-10',
       '2025-01-10 08:36:14.560000', '01/10/2025', '2025-01-13',
       '2025-01-13 08:47:09.020000', '2025-01-14',
       '2025-01-14 08:32:03.750000', '01/14/2025', '2025-01-15',
       '2025-01-15 08:24:31.860000', '01/15/2025',
       '2025-01-16 08:22:53.110000', '2025-01-16', '2025-01-11',
       '2025-01-17', '2025-01-17 08:49:15.690000'], dtype=object)

In [65]:
# i = 0
# for row in df['Date_Filed']:
#     print(str(i), row)
#     print(str_to_date(row))
#     i=i+1

In [66]:
df['Date Filed'] = df["Date Filed"].map(lambda x: str_to_date(x))

In [67]:
df['Date Filed'].unique()

array([datetime.date(2025, 1, 6), datetime.date(2025, 1, 2),
       datetime.date(2025, 1, 7), NaT, datetime.date(2025, 1, 8),
       datetime.date(2025, 1, 4), datetime.date(2025, 1, 9),
       datetime.date(2025, 1, 10), datetime.date(2025, 1, 13),
       datetime.date(2025, 1, 14), datetime.date(2025, 1, 15),
       datetime.date(2025, 1, 16), datetime.date(2025, 1, 11),
       datetime.date(2025, 1, 17)], dtype=object)

#### Date of Contract

In [68]:
df['Date of Contract'] = df["Date of Contract"].astype(str)

In [69]:
df['Date of Contract'] = df["Date of Contract"].map(lambda x: str_to_date(x))

In [70]:
df['Date of Contract'].unique()

array([datetime.date(2024, 12, 17), datetime.date(2025, 1, 3),
       datetime.date(2025, 1, 2), datetime.date(2025, 1, 6),
       datetime.date(2025, 1, 7), datetime.date(2025, 1, 8),
       datetime.date(2025, 1, 9), datetime.date(2025, 1, 10),
       datetime.date(2025, 12, 31), datetime.date(2025, 12, 23),
       datetime.date(2024, 12, 19), datetime.date(2025, 1, 13),
       datetime.date(2025, 1, 14), datetime.date(2025, 1, 15),
       datetime.date(2025, 1, 1), NaT, datetime.date(2025, 12, 17),
       datetime.date(2024, 1, 15), datetime.date(2025, 1, 16),
       datetime.date(2024, 7, 22), datetime.date(2024, 7, 11),
       datetime.date(2024, 8, 1), datetime.date(2024, 8, 16),
       datetime.date(2024, 9, 26), datetime.date(2024, 10, 18),
       datetime.date(2024, 11, 6), datetime.date(2024, 11, 19),
       datetime.date(2024, 12, 18), datetime.date(2025, 1, 17)],
      dtype=object)

In [71]:
df['Date of Contract'].dtype

dtype('O')

#### Contract ID Number

In [72]:
df['Contract ID Number'].unique()

array(['U240060424', 'U250000527', 'U250000530', 'BPN25TP0008',
       'VAL25PE0003', 'PEL24TS90051-3', 'PVP24TP90102-2',
       'PVP24TP90102-3', 'USA25TS90003-1', 'CHV02444PP', 'USV25PE0001',
       '8308551-1', '8309220-1', '8309304-1', '8309430-1',
       'GJX25TS998689', 250100.0, 250101.0, 250102.0, 196674.0,
       'BPP25TP90005-1', 'MAB25TS90004-1', 'VAL25TP90003-1',
       'VAL25TP90004-1', 'BPP25TS90006-1', 'IDF25TS90002-1', 'U250000693',
       'U250000694', 'U250000910', 'U250000946', 'TARTA25PE998755',
       'TARTA25TS998752', 'MAR25TP0005', 'PBF25PE0001', 'PBF25TP0003',
       'CHE25PE0003', 'PEK25PE0001', 'BBENC25PE0002', 'MIECA25PE0001',
       'SHLTR25PE0001', 'TARTA25PE0003', 'VALMS25TP0025', '8311181-1',
       '8312047-1', '8312723-1', '8313204-1', '8313206-1', '8313994-1',
       '8314155-1', '8314163-1', '8314196-1', '8314227-1', '8314255-1',
       '8314266-1', '8315017-1', '8315053-1', '8315058-1', '8315605-1',
       'CHV02446PP', 'PETEU25TS0001', 'PHILP25TP00

In [73]:
df['Contract ID Number'] = df['Contract ID Number'].astype(str)

In [74]:
df['Contract ID Number'].unique()

array(['U240060424', 'U250000527', 'U250000530', 'BPN25TP0008',
       'VAL25PE0003', 'PEL24TS90051-3', 'PVP24TP90102-2',
       'PVP24TP90102-3', 'USA25TS90003-1', 'CHV02444PP', 'USV25PE0001',
       '8308551-1', '8309220-1', '8309304-1', '8309430-1',
       'GJX25TS998689', '250100.0', '250101.0', '250102.0', '196674.0',
       'BPP25TP90005-1', 'MAB25TS90004-1', 'VAL25TP90003-1',
       'VAL25TP90004-1', 'BPP25TS90006-1', 'IDF25TS90002-1', 'U250000693',
       'U250000694', 'U250000910', 'U250000946', 'TARTA25PE998755',
       'TARTA25TS998752', 'MAR25TP0005', 'PBF25PE0001', 'PBF25TP0003',
       'CHE25PE0003', 'PEK25PE0001', 'BBENC25PE0002', 'MIECA25PE0001',
       'SHLTR25PE0001', 'TARTA25PE0003', 'VALMS25TP0025', '8311181-1',
       '8312047-1', '8312723-1', '8313204-1', '8313206-1', '8313994-1',
       '8314155-1', '8314163-1', '8314196-1', '8314227-1', '8314255-1',
       '8314266-1', '8315017-1', '8315053-1', '8315058-1', '8315605-1',
       'CHV02446PP', 'PETEU25TS0001', 'PHI

In [75]:
def drop_decimal(string):
    regex = '(\.)(0)$'
    match = re.search(regex, string)
    if match: 
        return string.replace('.0','')
    else: return string

In [76]:
string = 'BBE24PE0020'
drop_decimal(string)

'BBE24PE0020'

In [77]:
df['Contract ID Number'] = df["Contract ID Number"].map(lambda string: drop_decimal(string))

In [78]:
df['Contract ID Number'].unique()

array(['U240060424', 'U250000527', 'U250000530', 'BPN25TP0008',
       'VAL25PE0003', 'PEL24TS90051-3', 'PVP24TP90102-2',
       'PVP24TP90102-3', 'USA25TS90003-1', 'CHV02444PP', 'USV25PE0001',
       '8308551-1', '8309220-1', '8309304-1', '8309430-1',
       'GJX25TS998689', '250100', '250101', '250102', '196674',
       'BPP25TP90005-1', 'MAB25TS90004-1', 'VAL25TP90003-1',
       'VAL25TP90004-1', 'BPP25TS90006-1', 'IDF25TS90002-1', 'U250000693',
       'U250000694', 'U250000910', 'U250000946', 'TARTA25PE998755',
       'TARTA25TS998752', 'MAR25TP0005', 'PBF25PE0001', 'PBF25TP0003',
       'CHE25PE0003', 'PEK25PE0001', 'BBENC25PE0002', 'MIECA25PE0001',
       'SHLTR25PE0001', 'TARTA25PE0003', 'VALMS25TP0025', '8311181-1',
       '8312047-1', '8312723-1', '8313204-1', '8313206-1', '8313994-1',
       '8314155-1', '8314163-1', '8314196-1', '8314227-1', '8314255-1',
       '8314266-1', '8315017-1', '8315053-1', '8315058-1', '8315605-1',
       'CHV02446PP', 'PETEU25TS0001', 'PHILP25TP00

In [79]:
def assign_none(string):
    if string is None: return None
    if len(string)>=6: return string
    else: return None

In [80]:
df['Contract ID Number'] = df["Contract ID Number"].map(lambda string: assign_none(string))

In [81]:
df['Contract ID Number'].unique()

array(['U240060424', 'U250000527', 'U250000530', 'BPN25TP0008',
       'VAL25PE0003', 'PEL24TS90051-3', 'PVP24TP90102-2',
       'PVP24TP90102-3', 'USA25TS90003-1', 'CHV02444PP', 'USV25PE0001',
       '8308551-1', '8309220-1', '8309304-1', '8309430-1',
       'GJX25TS998689', '250100', '250101', '250102', '196674',
       'BPP25TP90005-1', 'MAB25TS90004-1', 'VAL25TP90003-1',
       'VAL25TP90004-1', 'BPP25TS90006-1', 'IDF25TS90002-1', 'U250000693',
       'U250000694', 'U250000910', 'U250000946', 'TARTA25PE998755',
       'TARTA25TS998752', 'MAR25TP0005', 'PBF25PE0001', 'PBF25TP0003',
       'CHE25PE0003', 'PEK25PE0001', 'BBENC25PE0002', 'MIECA25PE0001',
       'SHLTR25PE0001', 'TARTA25PE0003', 'VALMS25TP0025', '8311181-1',
       '8312047-1', '8312723-1', '8313204-1', '8313206-1', '8313994-1',
       '8314155-1', '8314163-1', '8314196-1', '8314227-1', '8314255-1',
       '8314266-1', '8315017-1', '8315053-1', '8315058-1', '8315605-1',
       'CHV02446PP', 'PETEU25TS0001', 'PHILP25TP00

#### Position Sequence Number

In [82]:
df['Position Sequence Number'].unique()

array([2.0, 1.0, 'n/a', '', 'unavailable', '(Empty)', 'N/A', 3.0, 'NA',
       4.0, 'NOT APPLICABLE'], dtype=object)

In [83]:
df['Position Sequence Number'] = df['Position Sequence Number'].astype(str)
df["Position Sequence Number"] = df["Position Sequence Number"].map(lambda x: x.replace('.0', ''))

In [84]:
dict_Directory['Position Seq. Num.']

{'No Info.': ['(Empty)', 'nan', 'unavailable']}

In [85]:
dict_Directory.keys()

dict_keys(['Company', 'Position Seq. Num.', 'CPID', 'Counterparty Contact', 'TL', 'Sell_Buy', 'Product Type', 'Product', 'Delivery Method', 'Delivery Location', 'Price Basis Method', 'Reference Product'])

In [86]:
key = list(dict_Directory['Position Seq. Num.'].keys())[0]
key

'No Info.'

In [87]:
values = list(dict_Directory['Position Seq. Num.'].values())[0]
values

['(Empty)', 'nan', 'unavailable']

In [88]:
df["Position Sequence Number"] = df["Position Sequence Number"].replace(values, key)
df['Position Sequence Number'].unique()

array(['2', '1', 'n/a', '', 'No Info.', 'N/A', '3', 'NA', '4',
       'NOT APPLICABLE'], dtype=object)

In [89]:
def get_pos_seq_number(string):
    try: 
        if str(int(string)).isnumeric(): return string
    except: return 'No Info.'   

In [90]:
string = '1'
get_pos_seq_number(string)

'1'

In [91]:
string = 'N/A'
get_pos_seq_number(string)

'No Info.'

In [92]:
df["Position Sequence Number"] = df["Position Sequence Number"].map(lambda string: get_pos_seq_number(string))

In [93]:
df['Position Sequence Number'].unique()

array(['2', '1', 'No Info.', '3', '4'], dtype=object)

#### CPID

In [94]:
df['CPID'] = df["CPID"].astype(str)

In [95]:
df["CPID"].unique()

array(['U240060424-2', 'U250000527-1', 'U250000530-1', '', 'n/a',
       '30968860.0', 'NA', 'unavailable', '(Empty)', 'N/A',
       'U250000693-1', 'U250000694-1', 'U250000910-1', 'U250000910-2',
       'U250000946-1', '1.0', '30989700.0', 'U250001295-1', '8571959.1',
       '8572055.1', '30995248.0', '30996647.0', '30996144.0',
       'U250001382-1', 'U250001611-1', 'U250001591-1', 'U250001589-1',
       'U250001588-1', 'U250001587-1', 'U250001481-1', 'U250001382-2',
       '31001382.0', '2.0', 'U250001694-1', 'U250001777-1',
       'U250001801-1', 'U250001855-1', 'U250001867-1', 'U250001869-1',
       'U250001889-1', '31015249.0', '31015250.0', 'PC1107203',
       'PC1107204', '8580060.1', 'U250002097-1', 'PC1107230',
       'U250002387-1', 'U250002389-1', 'U250002436-1', '8591154.1', '3.0',
       'U250002570-1', 'U250002571-1', 'U250002572-1', 'U250002586-1',
       'U250002586-2', 'U250002588-1', 'U250002616-1', 'U250002767-1',
       'U250002767-2', 'U250002767-3', 'U250002767-4

In [96]:
dict_Directory['CPID']

{'No Info.': ['(Empty)', 'nan', 'unavailable']}

In [97]:
df['CPID'] = df['CPID'].astype(str)
df["CPID"] = df["CPID"].map(lambda x: x.replace('.0', ''))

In [98]:
key = list(dict_Directory['CPID'].keys())[0]
key

'No Info.'

In [99]:
values = list(dict_Directory['CPID'].values())[0]
values

['(Empty)', 'nan', 'unavailable']

In [100]:
df["CPID"] = df["CPID"].replace(values, key)
df['CPID'].unique()

array(['U240060424-2', 'U250000527-1', 'U250000530-1', '', 'n/a',
       '30968860', 'NA', 'No Info.', 'N/A', 'U250000693-1',
       'U250000694-1', 'U250000910-1', 'U250000910-2', 'U250000946-1',
       '1', '30989700', 'U250001295-1', '8571959.1', '8572055.1',
       '30995248', '30996647', '30996144', 'U250001382-1', 'U250001611-1',
       'U250001591-1', 'U250001589-1', 'U250001588-1', 'U250001587-1',
       'U250001481-1', 'U250001382-2', '31001382', '2', 'U250001694-1',
       'U250001777-1', 'U250001801-1', 'U250001855-1', 'U250001867-1',
       'U250001869-1', 'U250001889-1', '31015249', '31015250',
       'PC1107203', 'PC1107204', '8580060.1', 'U250002097-1', 'PC1107230',
       'U250002387-1', 'U250002389-1', 'U250002436-1', '8591154.1', '3',
       'U250002570-1', 'U250002571-1', 'U250002572-1', 'U250002586-1',
       'U250002586-2', 'U250002588-1', 'U250002616-1', 'U250002767-1',
       'U250002767-2', 'U250002767-3', 'U250002767-4', '8597794.1',
       '8597798.1', '310998

#### Trading Counterparty Entity

In [101]:
def find_key_with_value(input_dict, value):
    for key, val in input_dict.items():
        if val == value: return key
    return "None"

def find_key_in_list(nokey, key_list):
        
    dict_ = {}
    for key in key_list:
        # check if key.upper() is inside nokey.upper()
        dict_[key] = round(fuzz.token_set_ratio(key.upper(), nokey.upper()),2)

    return find_key_with_value(dict_, 100)

In [102]:
for key, values in dict_Directory['Company'].items():
    df["Trading Counterparty Entity"] = df["Trading Counterparty Entity"].replace(values, key)

In [103]:
set(df["Trading Counterparty Entity"].unique()) <= set(dict_Directory['Company'].keys())

False

In [104]:
# Nombres de company que no estan en keys
nokey_list = list(set(df["Trading Counterparty Entity"].unique()) - set(dict_Directory['Company'].keys()))
nokey_list

['TARTAN OIL LLC',
 'Shell Oil',
 'Offen Petroleum',
 'Marathon Petroleum',
 'Mieco LLC',
 'US VENTURE',
 'US OIL AND REFINING CO-LE',
 'BP North America Petrol., a div.of BPPNA',
 'Petro-Diamond Incorporated-SDP',
 'CHEVRON PRODUCTS COMPANY-LE',
 'PETROCOM ENERGY GROUP LLC',
 'OFFEN PETROLEUM, LLC',
 'REG Marketing & Logistics Group, LLC',
 'IDEMITSU APOLLO CORPORATION',
 'PETRO-DIAMOND INCORPORATED',
 'VALERO MKT AND SUPPLY-LE',
 'PHILLIPS 66 COMPANY-LE',
 'BP NA PETROLEUM',
 'SBR MARKETING LLC',
 'QT FUELS INCORPORATED',
 'GRESHAM PETROLEUM COMPANY',
 'Parkland (U.S.) Supply Corporation',
 'Chevron Prod. Co, A Div.of Chev. USA, Inc.',
 'PHILLIPS 66 COMPANY',
 'SHELL TRADING (US) COMPANY',
 'CHEVRON PRODUCTS COMPANY',
 'PBF HOLDING COMPANY LLC-LE',
 'GENERAL PETROLEUM LLCDBA PTL MARINE',
 'IDEMITSU APOLLO CORP',
 'Mieco LLC.',
 'MARATHON PETROLEUM CO-LE',
 'NGL CRUDE LOGISTICS, LLC',
 'Brad Hall',
 'HF SINCLAIR REFINING & MARKETING LLC',
 'MARATHON PETROLEUM COMPANY LP',
 'IDEMITSU A

In [105]:
# Key values
key_list = list(dict_Directory['Company'].keys())
key_list

['US Venture',
 'Alaska Airlines',
 'Allegiant Air',
 'Altair',
 'Ampol',
 'Aramco',
 'Archer-Daniels',
 'Avfuel',
 'BB',
 'Biosphere',
 'Boyett',
 'BP',
 'Castleton',
 'CCMT',
 'Chase Inc',
 'Chevron',
 'Clean2050',
 'CMA CGM',
 'Cockett',
 'Crimson',
 'DAN Bunkering',
 'Diamond Green',
 'Eastern Aviation',
 'Elbow River',
 'ENEOS',
 'Enpramex',
 'Epsilon',
 'Equilon',
 'Flyers',
 'FSM',
 'General Pet.',
 'Glander Int.',
 'Glencore',
 'GreenAmerica',
 'Gresham',
 'Gunvor',
 'Hartland',
 'Herbst',
 'HF Sinclair',
 'HTP',
 'Idemitsu',
 'JACO',
 'Jaco Oil',
 'Kern',
 'Keyera',
 'Kinder Morgan',
 'KPI',
 'Lincoln Oil',
 'Mansfield',
 'Marathon',
 'Martinez Renewables',
 'Maverik',
 'McCall',
 'Mieco',
 'Monjasa',
 'Murex',
 'Musket',
 'n/a',
 'NAFCO',
 'Neste',
 'NGL',
 'N.A.Fuels',
 'Offen',
 'P66',
 'Parkland',
 'PBF',
 'Petro Diamond',
 'Petro Star',
 'Petrochina',
 'Petrocom',
 'Pilot Travel',
 'QuickTrip',
 'QT Fuels',
 'Rebel Oil',
 'REG Mktg.',
 'RIL USA',
 'River City',
 'Rubis',


In [106]:
# Fuzzy match 
dictio_ = {}
for nokey in nokey_list:
    print(nokey)
    dict_ = {}
    for key in key_list:
        value_list = list(dict_Directory['Company'][key])
        dict_[key] = np.max(np.array([round(fuzz.token_set_ratio(value.upper(), nokey.upper()),2) for value in value_list]))
            
    key_max = max(dict_, key= lambda x: dict_[x])
    print(key_max)
    dictio_[nokey] = key_max

TARTAN OIL LLC
Tartan
Shell Oil
Shell
Offen Petroleum
Offen
Marathon Petroleum
Marathon
Mieco LLC
Mieco
US VENTURE
US Venture
US OIL AND REFINING CO-LE
US Oil
BP North America Petrol., a div.of BPPNA
BP
Petro-Diamond Incorporated-SDP
Petro Diamond
CHEVRON PRODUCTS COMPANY-LE
Chevron
PETROCOM ENERGY GROUP LLC
Petrocom
OFFEN PETROLEUM, LLC
Offen
REG Marketing & Logistics Group, LLC
REG Mktg.
IDEMITSU APOLLO CORPORATION
Idemitsu
PETRO-DIAMOND INCORPORATED
Petro Diamond
VALERO MKT AND SUPPLY-LE
Valero
PHILLIPS 66 COMPANY-LE
P66
BP NA PETROLEUM
BP
SBR MARKETING LLC
SBR Mktg.
QT FUELS INCORPORATED
QuickTrip
GRESHAM PETROLEUM COMPANY
Gresham
Parkland (U.S.) Supply Corporation
Parkland
Chevron Prod. Co, A Div.of Chev. USA, Inc.
Chevron
PHILLIPS 66 COMPANY
P66
SHELL TRADING (US) COMPANY
Shell
CHEVRON PRODUCTS COMPANY
Chevron
PBF HOLDING COMPANY LLC-LE
PBF
GENERAL PETROLEUM LLCDBA PTL MARINE
General Pet.
IDEMITSU APOLLO CORP
Idemitsu
Mieco LLC.
Mieco
MARATHON PETROLEUM CO-LE
Marathon
NGL CRUDE L

In [107]:
dictio_

{'TARTAN OIL LLC': 'Tartan',
 'Shell Oil': 'Shell',
 'Offen Petroleum': 'Offen',
 'Marathon Petroleum': 'Marathon',
 'Mieco LLC': 'Mieco',
 'US VENTURE': 'US Venture',
 'US OIL AND REFINING CO-LE': 'US Oil',
 'BP North America Petrol., a div.of BPPNA': 'BP',
 'Petro-Diamond Incorporated-SDP': 'Petro Diamond',
 'CHEVRON PRODUCTS COMPANY-LE': 'Chevron',
 'PETROCOM ENERGY GROUP LLC': 'Petrocom',
 'OFFEN PETROLEUM, LLC': 'Offen',
 'REG Marketing & Logistics Group, LLC': 'REG Mktg.',
 'IDEMITSU APOLLO CORPORATION': 'Idemitsu',
 'PETRO-DIAMOND INCORPORATED': 'Petro Diamond',
 'VALERO MKT AND SUPPLY-LE': 'Valero',
 'PHILLIPS 66 COMPANY-LE': 'P66',
 'BP NA PETROLEUM': 'BP',
 'SBR MARKETING LLC': 'SBR Mktg.',
 'QT FUELS INCORPORATED': 'QuickTrip',
 'GRESHAM PETROLEUM COMPANY': 'Gresham',
 'Parkland (U.S.) Supply Corporation': 'Parkland',
 'Chevron Prod. Co, A Div.of Chev. USA, Inc.': 'Chevron',
 'PHILLIPS 66 COMPANY': 'P66',
 'SHELL TRADING (US) COMPANY': 'Shell',
 'CHEVRON PRODUCTS COMPANY': '

In [108]:
df["Trading Counterparty Entity"] = df["Trading Counterparty Entity"].replace(dictio_)

In [109]:
# Nombres de company que no estan en keys
nokey_list = list(set(df["Trading Counterparty Entity"].unique()) - set(dict_Directory['Company'].keys()))
nokey_list

[]

In [110]:
df["Trading Counterparty Entity"].unique()

array(['Trafigura', 'Sun Valley', 'BP', 'Valero', 'Petro Diamond', 'PBF',
       'US Venture', 'Chevron', 'Marathon', 'Tartan', 'Glencore', 'P66',
       'Idemitsu', 'BB', 'Mieco', 'Shell', 'Flyers', 'Offen', 'Vitol',
       'Musket', 'Jankovich', 'Parkland', 'QuickTrip', 'General Pet.',
       'Petrocom', 'TAC', 'HF Sinclair', 'McCall', 'REG Mktg.', 'HTP',
       'Hartland', 'QT Fuels', 'Tauber', 'SBR Mktg.', 'Epsilon',
       'US Energy', 'Gresham', 'US Oil', 'Rebel Oil', 'NGL', 'Enpramex',
       'Boyett'], dtype=object)

#### Trading Counterparty Entity Contact Name

In [111]:
df["Trading Counterparty Entity Contact Name"].unique()

array(['Danny Wilson', 'Allen Nivens', 'Michael Kinninger',
       'Travis Harper', 'n/a', '-', 'Ron Dunner', 'unavailable',
       'Bryant, Dennis', 'Aaron Ebers', 'Mike Kinniger', 'Jessica Terry',
       'Lou Forman', 'Alex Lerullo', 'Nguyen, Nemo', 'Matthew Stachel',
       'Justin Ladanza', 'Stuart Wilson', 'Ed McMonigle', 'Chris Lee',
       'Rick Anchetta', 'Michael Golen', 'Arturo Rendon',
       'Matthew Priesmeyer', 'Pascu, Chelsea', 'Ebbers, Aaron',
       'Harper, Travis', 'Alex Ierullo', 'McMonigle, Ed', 'Marino, Jeff',
       'Mohtashami, Steve', 'Frost, Nolan', 'Jeff Rogers', '',
       'Claudia Armstrong', 'Rick Ancheta', 'Nick Kerkhoff',
       'Vincent Dusina', 'Keith Leek', 'Allison Robinson',
       'Audrey Nelson', 'Rendon, Arturo', 'Wickman, Casey',
       'Spomer, Rick', 'Garrity, Matt', 'Gobee, Peter', 'Hill, Greg',
       'Greg', 'Matthew Garrity', 'Matt Garrity', 'Nolan Frost',
       'Wilson, Stuart', 'Kim Kuchenbecker', 'Blake Hale', 'Mark Resnick',
       'C

#### Spot Market Trading Location

In [112]:
dict_Directory['TL']

{'LA': ['#E101 (LOS ANGELES AREA) DEFAULT',
  'Bradshaw',
  'BTB Los Angeles CA Kinder Morgan',
  'Carson CA KM Watson Station',
  'CARSON, CA - KINDER MORGAN',
  'CARSON, CA - SOPUS',
  'Colton',
  'DOLORES-TESORO',
  'El Centro, CA',
  'GEORGETOWN, CAYMAN ISLANDS',
  'HARPERTOWN-KERN OIL',
  'Imperial',
  'LA',
  'LA,CA',
  'Las Vegas',
  'Los Angeles',
  'Los Angeles CA Kinder Morgan',
  'Los Angeles CA LAX Airport Terminal',
  'Mobile',
  'N/A',
  'Olympus',
  'Phoenix',
  'San Diego CA KM Mission Valley Terminal',
  'UNK',
  'Unmapped',
  'WILMINGTON-TESORO',
  'Zenith',
  ''],
 'SF': ['BAY',
  'Bay Area',
  'Concord CA KM PL Station',
  'CONCORD, CA - KM PUMPSTATION',
  'Martinez CA Avon Wharf',
  'MARTINEZ, CA - TRANSMONTAIGNE',
  'Mission Valley',
  'OAKLAND INTL. AIRPORT OAK, CA',
  'Reno',
  'RICHMOND, CA - CHEVRON',
  'RICHMOND, CA - IMTT',
  'RICHMOND-CHEVRON',
  'SACRAMENTO, CA - KINDER MORGAN',
  'San Francisco',
  'SAN FRANCISCO AIRPORT SFO, CA',
  'San Francisco CA Kind

In [113]:
df['Spot Market Trading Location'].unique()

array(['SF', 'LA', None, 'BAY', 'Portland - KM', 'Los Angeles',
       'West Coast', 'LOS ANGELES, CA', '', 'Portland - KMW', 'Imperial',
       'Phoenix', 'Conventional Regular', 'Conventional Premium',
       'ULSD #2', 'Portland-McCall', 'Las Vegas', 'LOS ANGELES',
       'Los Angeles, CA'], dtype=object)

In [114]:
for key, values in dict_Directory['TL'].items():
    df['Spot Market Trading Location'] = df['Spot Market Trading Location'].replace(values, key)

In [115]:
set(df['Spot Market Trading Location'].unique()) <= set(dict_Directory['TL'].keys())

False

In [116]:
#if not set(df["Trading Counterparty Entity"].unique()) <= set(dict_Directory['Company'].keys()):
# Nombres de company que no estan en keys
nokey_list = list(set(df['Spot Market Trading Location'].unique()) - set(dict_Directory['TL'].keys()))
nokey_list = [str(nokey) for nokey in nokey_list]

if "nan" in nokey_list:
    nokey_list.remove("nan")

nokey_list

['None',
 'Conventional Premium',
 'LOS ANGELES, CA',
 'Los Angeles, CA',
 'ULSD #2',
 'Conventional Regular',
 'West Coast',
 'LOS ANGELES']

In [117]:
key_list = list(dict_Directory['TL'].keys())
key_list

['LA', 'SF', 'NW']

In [118]:
dictio_ = {}
for nokey in nokey_list:
    print('nokey', nokey)
    dict_ = {}
    for key in key_list:
        print('key', key)
        value_list = list(dict_Directory['TL'][key])
        if len(value_list)==0: break
        print('value_list', value_list)
        dict_[key] = np.max(np.array([round(fuzz.token_set_ratio(value.upper(), nokey.upper()),2) for value in value_list]))
            
    key_max = max(dict_, key= lambda x: dict_[x])
    print(key_max)
    dictio_[nokey] = key_max

dictio_

nokey None
key LA
value_list ['#E101 (LOS ANGELES AREA) DEFAULT', 'Bradshaw', 'BTB Los Angeles CA Kinder Morgan', 'Carson CA KM Watson Station', 'CARSON, CA - KINDER MORGAN', 'CARSON, CA - SOPUS', 'Colton', 'DOLORES-TESORO', 'El Centro, CA', 'GEORGETOWN, CAYMAN ISLANDS', 'HARPERTOWN-KERN OIL', 'Imperial', 'LA', 'LA,CA', 'Las Vegas', 'Los Angeles', 'Los Angeles CA Kinder Morgan', 'Los Angeles CA LAX Airport Terminal', 'Mobile', 'N/A', 'Olympus', 'Phoenix', 'San Diego CA KM Mission Valley Terminal', 'UNK', 'Unmapped', 'WILMINGTON-TESORO', 'Zenith', '']
key SF
value_list ['BAY', 'Bay Area', 'Concord CA KM PL Station', 'CONCORD, CA - KM PUMPSTATION', 'Martinez CA Avon Wharf', 'MARTINEZ, CA - TRANSMONTAIGNE', 'Mission Valley', 'OAKLAND INTL. AIRPORT OAK, CA', 'Reno', 'RICHMOND, CA - CHEVRON', 'RICHMOND, CA - IMTT', 'RICHMOND-CHEVRON', 'SACRAMENTO, CA - KINDER MORGAN', 'San Francisco', 'SAN FRANCISCO AIRPORT SFO, CA', 'San Francisco CA Kinder Morgan', 'San Francisco CA KinderMorga', 'San Fra

{'None': 'SF',
 'Conventional Premium': 'LA',
 'LOS ANGELES, CA': 'LA',
 'Los Angeles, CA': 'LA',
 'ULSD #2': 'LA',
 'Conventional Regular': 'LA',
 'West Coast': 'LA',
 'LOS ANGELES': 'LA'}

In [119]:
df["Spot Market Trading Location"] = df["Spot Market Trading Location"].replace(dictio_)

# Nombres de company que no estan en keys
nokey_list = list(set(df["Spot Market Trading Location"].unique()) - set(dict_Directory['TL'].keys()))
print(nokey_list)

[None]


In [120]:
df['Spot Market Trading Location'].unique()

array(['SF', 'LA', None, 'NW'], dtype=object)

In [121]:
key_list = list(dict_Directory['TL'].keys())

In [122]:
def assign_none(string):
    if string is None: return None
    if string in key_list: return string
    else: return None

In [123]:
# df['Spot Market Trading Location'] = df['Spot Market Trading Location'].astype(str)
df["Spot Market Trading Location"] = df["Spot Market Trading Location"].map(lambda string: assign_none(string))

In [124]:
df['Spot Market Trading Location'].unique()

array(['SF', 'LA', None, 'NW'], dtype=object)

#### Seller or Buyer

In [125]:
df['Seller or Buyer'] = df['Seller or Buyer'].astype(str)

In [126]:
dict_Directory.keys()

dict_keys(['Company', 'Position Seq. Num.', 'CPID', 'Counterparty Contact', 'TL', 'Sell_Buy', 'Product Type', 'Product', 'Delivery Method', 'Delivery Location', 'Price Basis Method', 'Reference Product'])

In [127]:
dict_Directory['Sell_Buy']

{'Buy': ['B', 'Buy', 'Buyer'],
 'In-Tank Transfer': ['Buyer/Seller', 'Seller/Buyer'],
 'Sell': ['S', 'Sell', 'Seller']}

In [128]:
df['Seller or Buyer'].unique()

array(['Seller', 'Buyer', 'SELL', 'BUY', 'Buy', 'Sale', 'Purchase',
       'Sell', 'BUYER'], dtype=object)

In [129]:
for key, values in dict_Directory['Sell_Buy'].items():
    df['Seller or Buyer'] = df['Seller or Buyer'].replace(values, key)

set(df['Seller or Buyer'].unique()) <= set(dict_Directory['Sell_Buy'].keys())

False

In [130]:
#if not set(df["Trading Counterparty Entity"].unique()) <= set(dict_Directory['Company'].keys()):
# Nombres de company que no estan en keys
nokey_list = list(set(df['Seller or Buyer'].unique()) - set(dict_Directory['Sell_Buy'].keys()))
nokey_list = [str(nokey) for nokey in nokey_list]
nokey_list

['Sale', 'Purchase', 'BUY', 'SELL', 'BUYER']

In [131]:
key_list = list(dict_Directory['Sell_Buy'].keys())
key_list = list(map(str, key_list))
key_list = [clean_string(string) for string in key_list]
key_list = [clean_string(string) for string in key_list if string is not None]
key_list

['Buy', 'In-Tank Transfer', 'Sell']

In [132]:
dictio_ = {}
for nokey in nokey_list:
    print('nokey', nokey)
    dict_ = {}
    for key in key_list:
        print('key', key)
        value_list = list(dict_Directory['Sell_Buy'][key])
        print('value_list', value_list)
        dict_[key] = np.max(np.array([round(fuzz.token_set_ratio(value.upper(), nokey.upper()),2) for value in value_list]))
            
    key_max = max(dict_, key= lambda x: dict_[x])
    print(key_max)
    dictio_[nokey] = key_max

dictio_

nokey Sale
key Buy
value_list ['B', 'Buy', 'Buyer']
key In-Tank Transfer
value_list ['Buyer/Seller', 'Seller/Buyer']
key Sell
value_list ['S', 'Sell', 'Seller']
Sell
nokey Purchase
key Buy
value_list ['B', 'Buy', 'Buyer']
key In-Tank Transfer
value_list ['Buyer/Seller', 'Seller/Buyer']
key Sell
value_list ['S', 'Sell', 'Seller']
In-Tank Transfer
nokey BUY
key Buy
value_list ['B', 'Buy', 'Buyer']
key In-Tank Transfer
value_list ['Buyer/Seller', 'Seller/Buyer']
key Sell
value_list ['S', 'Sell', 'Seller']
Buy
nokey SELL
key Buy
value_list ['B', 'Buy', 'Buyer']
key In-Tank Transfer
value_list ['Buyer/Seller', 'Seller/Buyer']
key Sell
value_list ['S', 'Sell', 'Seller']
Sell
nokey BUYER
key Buy
value_list ['B', 'Buy', 'Buyer']
key In-Tank Transfer
value_list ['Buyer/Seller', 'Seller/Buyer']
key Sell
value_list ['S', 'Sell', 'Seller']
Buy


{'Sale': 'Sell',
 'Purchase': 'In-Tank Transfer',
 'BUY': 'Buy',
 'SELL': 'Sell',
 'BUYER': 'Buy'}

In [133]:
df['Seller or Buyer'] = df['Seller or Buyer'].replace(dictio_)

# Nombres de company que no estan en keys
nokey_list = list(set(df['Seller or Buyer'].unique()) - set(dict_Directory['Sell_Buy'].keys()))
nokey_list

[]

In [134]:
df['Seller or Buyer'].unique()

array(['Sell', 'Buy', 'In-Tank Transfer'], dtype=object)

#### Product Type

In [135]:
# Product type
# taking renewal diesel
# it should be diesel
# "Diesel"

In [136]:
df['Product Type'] = df["Product Type"].astype(str)

In [137]:
df['Product Type'].unique()

array(['Gasoline', 'Renewable Fuel', 'Renewable Fuels', 'Gasolines',
       'Diesel', 'Jet Fuel', 'Aviation Fuel', 'Diesel Fuel',
       'Diesel Fuels', 'Aviation fuel', 'Jet', 'ULSD', 'DIESEL',
       'Blending Components', 'GASOLINE', 'EPA Diesel'], dtype=object)

In [138]:
set(df['Product Type'].unique()) - set(dict_Directory['Product Type'].keys())

{'Aviation Fuel',
 'Blending Components',
 'DIESEL',
 'Diesel Fuel',
 'Diesel Fuels',
 'EPA Diesel',
 'GASOLINE',
 'Gasolines',
 'Jet',
 'Jet Fuel',
 'Renewable Fuels',
 'ULSD'}

In [139]:
dict_Directory['Product Type'].keys()

dict_keys(['Gasoline', 'Aviation fuel', 'Renewable Fuel', 'Blending Component', 'Diesel', 'Butane', 'Naphtha'])

In [140]:
dict_Directory['Product Type']['Aviation fuel']

['Aviation fuel', 'Jet', 'Jet Fuel']

In [141]:
def replace_string_dictio(string, dictio, str_old, str_new):
    string_ = str(string).lower().replace(str_old, str_new)
    dictio_ = {}
    for key, value in dictio.items(): 
        dictio_[key] = [str(v).lower().replace(str_old, str_new) for v in value] 
    return string_, dictio_

In [142]:
df['Product Type'].unique()

array(['Gasoline', 'Renewable Fuel', 'Renewable Fuels', 'Gasolines',
       'Diesel', 'Jet Fuel', 'Aviation Fuel', 'Diesel Fuel',
       'Diesel Fuels', 'Aviation fuel', 'Jet', 'ULSD', 'DIESEL',
       'Blending Components', 'GASOLINE', 'EPA Diesel'], dtype=object)

In [143]:
def aux_match(string):
    dictio = dict_Directory['Product Type'].copy()
    string_, dictio_ = replace_string_dictio(string, dictio, 'fuels', 'fuel')
    return match_dictio(string_, dictio_, case_sensitive=True)

In [144]:
df['Product Type'] = df["Product Type"].map(lambda x: aux_match(x))

In [145]:
df['Product Type'].unique()

array(['Gasoline', 'Renewable Fuel', 'Diesel', 'Aviation fuel',
       'Blending Component', None], dtype=object)

In [146]:
set(df['Product Type'].unique()) <= set(dict_Directory['Product Type'].keys())

False

In [147]:
#if not set(df["Trading Counterparty Entity"].unique()) <= set(dict_Directory['Company'].keys()):
# Nombres de company que no estan en keys
nokey_list = list(set(df['Product Type'].unique()) - set(dict_Directory['Product Type'].keys()))
# nokey_list = [str(nokey) for nokey in nokey_list]
nokey_list = [clean_string(string) for string in nokey_list if string is not None]
nokey_list

[]

In [148]:
key_list = list(map(str, key_list))
key_list = [clean_string(string) for string in key_list]
key_list = [clean_string(string) for string in key_list if string is not None]
key_list

['Buy', 'In-Tank Transfer', 'Sell']

In [149]:
key_list = list(dict_Directory['Product Type'].keys())
key_list

['Gasoline',
 'Aviation fuel',
 'Renewable Fuel',
 'Blending Component',
 'Diesel',
 'Butane',
 'Naphtha']

In [150]:
key_list = list(map(str, key_list))
key_list = [clean_string(string) for string in key_list]
key_list = [clean_string(string) for string in key_list if string is not None]
key_list

['Gasoline',
 'Aviation fuel',
 'Renewable Fuel',
 'Blending Component',
 'Diesel',
 'Butane',
 'Naphtha']

In [151]:
dictio_ = {}
for nokey in nokey_list:
    print(nokey)
    dict_ = {}
    for key in key_list:
        value_list = list(dict_Directory['Product Type'][key])
        dict_[key] = np.max(np.array([round(fuzz.token_set_ratio(value.upper(), nokey.upper()),2) for value in value_list]))
            
    key_max = max(dict_, key= lambda x: dict_[x])
    print(key_max)
    dictio_[nokey] = key_max

dictio_

{}

In [152]:
df['Product Type'] = df['Product Type'].replace(dictio_)

# Nombres de company que no estan en keys
nokey_list = list(set(df['Product Type'].unique()) - set(dict_Directory['Product Type'].keys()))
nokey_list

[None]

In [153]:
df['Product Type'].unique()

array(['Gasoline', 'Renewable Fuel', 'Diesel', 'Aviation fuel',
       'Blending Component', None], dtype=object)

#### Product Name

In [154]:
df['Product Name_'] = df["Product Name"].copy()

In [155]:
df['Product Name'] = df["Product Name"].astype(str)
df['Product Name'].unique()

array(['CBOB-R', 'CARBOB-R', 'Conventional', 'RD99', 'CARBOB REG 84',
       'AZBOB', 'R99', 'CARBOBUNL', 'ECA Marine Diesel', 'CARBOB Regular',
       'Conventional Regular', 'Jet A', 'JET A', 'EPA ULSD',
       'No 2 CARB ULSD 15', 'CARB ULSD', 'CARBOB', 'Renewable Diesel',
       'CARBULSD', 'CBOBUNL', 'ULSD', '84 Reg CARBOB', '89 Pre CARBOB',
       'LA CARBOB', '84 Reg SubCBG AZ', 'Jet Fuel', 'SF Bay Carbob',
       'LA EPA DSL', 'LA JET', 'CBOB', '#2 ULSD', 'Blendgrade 9-15#',
       'CBOB REG 84 H RVP FBW ETH 10%', 'CARBOB Reg 12.5#', 'CARBOB-P',
       'AZBOB-R', 'ULSD 2 CARB', 'ULSD 2', '#2DSL ULS 15',
       'RD99 HIGH FLASH MARINE D UNB', 'CBG AZ REG 87 L 7.0 RVP',
       'CARBOB PRE 88.5', 'Renewable Diesel ', 'CARBOBPRE', 'CBOBPRE',
       'LA EPA', 'Conventional Premium', 'ULSD #2', 'CARBOB Reg 5.99#',
       'CBOB PRE 88.5 H RVP FBW ETH 10%', 'SF EPA ULSD', 'CBOB Prm 13.5#',
       'No 2 ULSD 15', 'CARBOB Reg 14.0#', 'RBOB AZ REG 84', 'AZRBOBUNL',
       'Alkylate', 'Sub

In [156]:
df['Product Name'] = df["Product Name"].map(lambda string: clean_string(string))
df['Product Name'].unique()

array(['CBOB-R', 'CARBOB-R', 'Conventional', 'RD99', 'CARBOB REG 84',
       'AZBOB', 'R99', 'CARBOBUNL', 'ECA Marine Diesel', 'CARBOB Regular',
       'Conventional Regular', 'Jet A', 'JET A', 'EPA ULSD',
       'No 2 CARB ULSD 15', 'CARB ULSD', 'CARBOB', 'Renewable Diesel',
       'CARBULSD', 'CBOBUNL', 'ULSD', '84 Reg CARBOB', '89 Pre CARBOB',
       'LA CARBOB', '84 Reg SubCBG AZ', 'Jet Fuel', 'SF Bay Carbob',
       'LA EPA DSL', 'LA JET', 'CBOB', '#2 ULSD', 'Blendgrade 9-15#',
       'CBOB REG 84 H RVP FBW ETH 10%', 'CARBOB Reg 12.5#', 'CARBOB-P',
       'AZBOB-R', 'ULSD 2 CARB', 'ULSD 2', '#2DSL ULS 15',
       'RD99 HIGH FLASH MARINE D UNB', 'CBG AZ REG 87 L 7.0 RVP',
       'CARBOB PRE 88.5', 'Renewable Diesel ', 'CARBOBPRE', 'CBOBPRE',
       'LA EPA', 'Conventional Premium', 'ULSD #2', 'CARBOB Reg 5.99#',
       'CBOB PRE 88.5 H RVP FBW ETH 10%', 'SF EPA ULSD', 'CBOB Prm 13.5#',
       'No 2 ULSD 15', 'CARBOB Reg 14.0#', 'RBOB AZ REG 84', 'AZRBOBUNL',
       'Alkylate', 'Sub

In [157]:
dict_Directory['Product'].keys()

dict_keys(['CARBOB-R', 'ULSD No2', 'Jet-A', 'AZRBOB-R', 'AZRBOB-P', 'CBOB-R', 'Alkylate', 'Isomerate', 'REFORMATE', 'CARB No2', 'Renewable Diesel'])

In [158]:
dictio = dict_Directory['Product']
df_ = pd.DataFrame(df['Product Name'].unique())
df_.columns = ['Product Name']
df_ = df_[~df_['Product Name'].isna()]
df_['KEY CS=True'] = df_["Product Name"].map(lambda string: match_dictio(string, dictio, case_sensitive=True))
df_['KEY CS=False'] = df_["Product Name"].map(lambda string: match_dictio(string, dictio, case_sensitive=False))
df_

,Product Name,KEY CS=True,KEY CS=False
0,CBOB-R,CBOB-R,CBOB-R
1,CARBOB-R,CARBOB-R,CARBOB-R
2,Conventional,None,None
3,RD99,None,None
4,CARBOB REG 84,None,None
...,...,...,...
66,RBOB AZ PRE 91,None,None
67,AZRBOB PREMIUM,None,None
68,CONVENTIONAL,None,None
69,PREMIUM,None,None


In [159]:
dictio_ = {}
# dictio_max = {}
for nokey in df_['Product Name']:
    # print(nokey)
    dict_ = {}
    for key in list(dict_Directory['Product'].keys()):
        value_list = list(dict_Directory['Product'][key]) + [key]
        value_list = [str(value).strip() for value in value_list]
        # dict_[key] = np.max(np.array([round(fuzz.token_set_ratio(value.upper(), nokey.upper()),2) for value in value_list]))
        # dict_[key] = np.max(np.array([round(fuzz.ratio(value.upper(), nokey.upper()),2) for value in value_list]))
        # dict_[key] = np.max(np.array([round(fuzz.partial_ratio(value.upper(), nokey.upper()),2) for value in value_list]))
        dict_[key] = np.max(np.array([round(jaro_similarity(value.upper(), nokey.upper()),2) for value in value_list]))*100
        
    # key_max = max(dict_, key= lambda x: dict_[x])
    # dictio_[nokey] = key_max
    # dictio_max[nokey] = max(dict_.values())
    if max(dict_.values())>=70:
        dictio_[nokey] = [max(dict_, key= lambda x: dict_[x]), max(dict_.values())]

In [160]:
dict_ = {}
for key, value in dictio_.items():
    dict_[key] = value[0]

df_['KEY Fuzzy'] = df_['Product Name'].map(dict_).fillna("")
df_['KEY Fuzzy'] = df_["KEY Fuzzy"].map(lambda string: clean_string(string))
df_

,Product Name,KEY CS=True,KEY CS=False,KEY Fuzzy
0,CBOB-R,CBOB-R,CBOB-R,CBOB-R
1,CARBOB-R,CARBOB-R,CARBOB-R,CARBOB-R
2,Conventional,None,None,None
3,RD99,None,None,None
4,CARBOB REG 84,None,None,CARBOB-R
...,...,...,...,...
66,RBOB AZ PRE 91,None,None,AZRBOB-R
67,AZRBOB PREMIUM,None,None,AZRBOB-R
68,CONVENTIONAL,None,None,None
69,PREMIUM,None,None,None


In [161]:
dictio_true = dict(zip(df_['Product Name'], df_['KEY CS=True']))
dictio_true = {k: v for k, v in dictio_true.items() if v is not None}

In [162]:
dictio_false = dict(zip(df_['Product Name'], df_['KEY CS=False']))
dictio_false = {k: v for k, v in dictio_false.items() if v is not None}

In [163]:
dictio_Fuzzy = dict(zip(df_['Product Name'], df_['KEY Fuzzy']))
dictio_Fuzzy = {k: v for k, v in dictio_Fuzzy.items() if v is not None}

In [164]:
dictio = dict(dictio_Fuzzy)

for k, v in dictio.items():
    if k in list(dictio_false.keys()):
        dictio[k] = dictio_false[k]
    if k in list(dictio_true.keys()):
        dictio[k] = dictio_true[k]    

In [165]:
df['Product Name'] = df['Product Name'].replace(dictio)

In [166]:
(set(df['Product Name'].unique()) - {None}) <= set(dict_Directory['Product'].keys())

False

In [167]:
len((set(df['Product Name'].unique()) - {None}) - set(dict_Directory['Product'].keys()))

26

#### Contract Volume (Barrels)

In [168]:
df['Contract Volume'].unique()

array([5742.0, 5000.0, 5.0, 25.0, -150.0, 10.0, 15.0, -25.0, 50.0, 0.029,
       30.0, 25000.0, 100.0, 100000.0, 5500.0, 22.5, 1.0, 6.0, 6.8, 5.5,
       24999.9999999999, 1167.0, 27000.0, 12.0, -30.0, 200.0, 0.5, 14.0,
       10000.0, 2500.0, 125000.0, 62500.0, 50000.0, 17.6, 660.0, 450.0,
       6500.0, 40000.0, 31.25, 35.0, 62.5, 450000.0, 660000.0, 15000.0,
       -9.0, -10.0, 40.0, 4000.0, 2000.0, 3000.0, 1200.0, -5.0, 70.0,
       63.0, 42.0, 74.0, -15.0, -6.0, 42.5, 75.0, -100.0, -7.0,
       49999.9999999999, 8000.0, 6000.0, 11000.0, 30000.0, 20000.0,
       70000.0, 12000.0, 7.0, 4.0, 80.0, 300.0, 2.0, 3.5, 300000.0, 62.0,
       411.0, 43.0, 32.0, 13500.0, 11500.0, 14.5, -7.5, -35.0, -200.0,
       -20.0, 15075.0, 23000.0, 7000.0, 21000.0, 18000.0, 12500.0,
       16166.0, 17000.0, 16900.0, 16500.0, 13000.0, 16100.0, 14000.0,
       10500.0, 14500.0, 7.5, 20.0, 13.0], dtype=object)

In [169]:
df['Contract Volume'] = df["Contract Volume"].astype(str)

In [170]:
df["Contract Volume"].unique()

array(['5742.0', '5000.0', '5.0', '25.0', '-150.0', '10.0', '15.0',
       '-25.0', '50.0', '0.029', '30.0', '25000.0', '100.0', '100000.0',
       '5500.0', '22.5', '1.0', '6.0', '6.8', '5.5', '24999.9999999999',
       '1167.0', '27000.0', '12.0', '-30.0', '200.0', '0.5', '14.0',
       '10000.0', '2500.0', '125000.0', '62500.0', '50000.0', '17.6',
       '660.0', '450.0', '6500.0', '40000.0', '31.25', '35.0', '62.5',
       '450000.0', '660000.0', '15000.0', '-9.0', '-10.0', '40.0',
       '4000.0', '2000.0', '3000.0', '1200.0', '-5.0', '70.0', '63.0',
       '42.0', '74.0', '-15.0', '-6.0', '42.5', '75.0', '-100.0', '-7.0',
       '49999.9999999999', '8000.0', '6000.0', '11000.0', '30000.0',
       '20000.0', '70000.0', '12000.0', '7.0', '4.0', '80.0', '300.0',
       '2.0', '3.5', '300000.0', '62.0', '411.0', '43.0', '32.0',
       '13500.0', '11500.0', '14.5', '-7.5', '-35.0', '-200.0', '-20.0',
       '15075.0', '23000.0', '7000.0', '21000.0', '18000.0', '12500.0',
       '16166

In [171]:
df['Contract Volume'] = df["Contract Volume"].map(lambda x: extract_numbers(x))

In [172]:
df['Contract Volume'].unique()

array(['5742.0', '5000.0', '5.0', '25.0', '-150.0', '10.0', '15.0',
       '-25.0', '50.0', '0.029', '30.0', '25000.0', '100.0', '100000.0',
       '5500.0', '22.5', '1.0', '6.0', '6.8', '5.5', '24999.9999999999',
       '1167.0', '27000.0', '12.0', '-30.0', '200.0', '0.5', '14.0',
       '10000.0', '2500.0', '125000.0', '62500.0', '50000.0', '17.6',
       '660.0', '450.0', '6500.0', '40000.0', '31.25', '35.0', '62.5',
       '450000.0', '660000.0', '15000.0', '-9.0', '-10.0', '40.0',
       '4000.0', '2000.0', '3000.0', '1200.0', '-5.0', '70.0', '63.0',
       '42.0', '74.0', '-15.0', '-6.0', '42.5', '75.0', '-100.0', '-7.0',
       '49999.9999999999', '8000.0', '6000.0', '11000.0', '30000.0',
       '20000.0', '70000.0', '12000.0', '7.0', '4.0', '80.0', '300.0',
       '2.0', '3.5', '300000.0', '62.0', '411.0', '43.0', '32.0',
       '13500.0', '11500.0', '14.5', '-7.5', '-35.0', '-200.0', '-20.0',
       '15075.0', '23000.0', '7000.0', '21000.0', '18000.0', '12500.0',
       '16166

In [173]:
df['Contract Volume'] = df['Contract Volume'].astype(float)

In [174]:
np.set_printoptions(suppress=True)
df['Contract Volume'].unique()

array([  5742.   ,   5000.   ,      5.   ,     25.   ,   -150.   ,
           10.   ,     15.   ,    -25.   ,     50.   ,      0.029,
           30.   ,  25000.   ,    100.   , 100000.   ,   5500.   ,
           22.5  ,      1.   ,      6.   ,      6.8  ,      5.5  ,
        25000.   ,   1167.   ,  27000.   ,     12.   ,    -30.   ,
          200.   ,      0.5  ,     14.   ,  10000.   ,   2500.   ,
       125000.   ,  62500.   ,  50000.   ,     17.6  ,    660.   ,
          450.   ,   6500.   ,  40000.   ,     31.25 ,     35.   ,
           62.5  , 450000.   , 660000.   ,  15000.   ,     -9.   ,
          -10.   ,     40.   ,   4000.   ,   2000.   ,   3000.   ,
         1200.   ,     -5.   ,     70.   ,     63.   ,     42.   ,
           74.   ,    -15.   ,     -6.   ,     42.5  ,     75.   ,
         -100.   ,     -7.   ,  50000.   ,   8000.   ,   6000.   ,
        11000.   ,  30000.   ,  20000.   ,  70000.   ,  12000.   ,
            7.   ,      4.   ,     80.   ,    300.   ,      2.

In [175]:
# Product Type
['Gasoline', 'Aviation fuel', 'Renewable Fuel', 'Blending Component', 'Diesel', 'Butane', 'Naphtha']

['Gasoline',
 'Aviation fuel',
 'Renewable Fuel',
 'Blending Component',
 'Diesel',
 'Butane',
 'Naphtha']

In [176]:
df['Product Type'].unique()

array(['Gasoline', 'Renewable Fuel', 'Diesel', 'Aviation fuel',
       'Blending Component', None], dtype=object)

In [177]:
dict_Directory['Product Type'].keys()

dict_keys(['Gasoline', 'Aviation fuel', 'Renewable Fuel', 'Blending Component', 'Diesel', 'Butane', 'Naphtha'])

In [178]:
# df['B'] = np.where( If , Then , Else )
df['Contract Volume'] = np.where(np.logical_and(df['Product Type']!='Blending Component', np.abs(df['Contract Volume'])>300), 
                                           df['Contract Volume']/1000, 
                                           df['Contract Volume'])

In [179]:
df['Contract Volume'].unique()

array([   5.742,    5.   ,   25.   , -150.   ,   10.   ,   15.   ,
        -25.   ,   50.   ,    0.029,   30.   ,  100.   ,    5.5  ,
         22.5  ,    1.   ,    6.   ,    6.8  ,   25.   ,    1.167,
         27.   ,   12.   ,  -30.   ,  200.   ,    0.5  ,   14.   ,
          2.5  ,  125.   ,   62.5  ,   17.6  ,    0.66 ,    0.45 ,
          6.5  ,   40.   ,   31.25 ,   35.   ,  450.   ,  660.   ,
         -9.   ,  -10.   ,    4.   ,    2.   ,    3.   ,    1.2  ,
         -5.   ,   70.   ,   63.   ,   42.   ,   74.   ,  -15.   ,
         -6.   ,   42.5  ,   75.   , -100.   ,   -7.   ,   50.   ,
          8.   ,   11.   ,   20.   ,    7.   ,   80.   ,  300.   ,
          3.5  ,   62.   ,    0.411,   43.   ,   32.   ,   13.5  ,
         11.5  ,   14.5  ,   -7.5  ,  -35.   , -200.   ,  -20.   ,
         15.075,   23.   ,   21.   ,   18.   ,   12.5  ,   16.166,
         17.   ,   16.9  ,   16.5  ,   13.   ,   16.1  ,   10.5  ,
          7.5  ])

#### Contract Delivery Month

In [180]:
df['Contract Delivery Month'] = df["Contract Delivery Month"].astype(str)

In [181]:
df['Contract Delivery Month'].unique()

array(['2025-01-18 06:00:00', '2025-02-04 06:00:00',
       '2025-03-07 06:00:00', '2025-01-31', '2024-12-27', '2025-01-25',
       '2025-01-08', '2025-01-01', '01/31/2025', '01/03/2025',
       '2025-01-17', 'Jan', 'Feb', '2025-02-25', '2025-02-14',
       '2025-01-16', '2025-01-18', '2025-01-06', '2025-01-19',
       '2025-02-28', '2025-02-01', '02/28/2025', '01/07/2025',
       '01/10/2025', '2025-01-19 06:00:00', '2025-01-20', '2025-03-25',
       '2025-01-07', '01/01/2025', '02/01/2025', '2025-06-25',
       '2025-05-25', '2025-04-25', '2025-03-01', '2025-06-15',
       '2025-05-16', '2025-04-15', '2025-03-31', '2025-01-23',
       '2025-04-01', '2025-05-01', '2025-06-01', '2025-03-16',
       '01/09/2025', 'Mar', 'Dec', '2025-01-14', '2025-02-14 06:00:00',
       '2025-01-22 06:00:00', '01/13/2025', '01/23/2025', '2025-01-22',
       '2025-01-24', '2025-01-27 06:00:00', '2025-01-24 06:00:00',
       '2025-01-25 06:00:00', '2025-01-26 06:00:00',
       '2025-01-29 06:00:00', '2025

In [182]:
# for string in df['Contract Delivery Month']:
#     print(string)
#     str_to_month(string)

In [183]:
df['Contract Delivery Month'] = df["Contract Delivery Month"].map(lambda string: str_to_month(string))

In [184]:
df['Contract Delivery Month'].unique()

array(['Jan-2025', 'Feb-2025', 'Mar-2025', 'Dec-2024', 'Jun-2025',
       'May-2025', 'Apr-2025', 'Dec-2025', None, 'Aug-2025', 'Jul-2025',
       'Sep-2025', 'Oct-2025', 'Nov-2025'], dtype=object)

#### Delivery Method

In [185]:
df['Delivery Method'].unique()

array(['Pipeline', 'In-Tank Transfer', 'Kinder Morgan (West Line)',
       'In Store Transfer', 'Barge', 'Head of Pipe ', 'In-tank',
       'in-tank ',
       'by Pipeline, Pumpover, Book, In-Tank Transfer, or PTO as mutually agreed',
       'by Pipeline or Book Trnsfr as mutually agreed', 'Bulk', 'Pipe',
       'Pipe Only', 'Other (specify) - Bulk',
       'in-tank transfer/ pipeline', 'Tank Transfer',
       'Kinder Morgan (North Line)', 'Delivered', 'Other (Railcar)',
       'Marine Vessel', 'PIPELINE', 'Vessel', 'Pump-Over', 'Marine',
       'Other (As Directed)', 'In Tank Transfer',
       'By Pipeline,Book Trnsfr,In-tank Trnsfr,or Allocation Trnsfr as mutually agreed',
       'by Pipeline,Book Trnsfr,Pumpovr,Barge,Vessel or Intnk Trnsfr as mutually agreed',
       'Kinder Morgan (SF Bay Line)', 'Other (Truck)'], dtype=object)

In [186]:
df['Delivery Method'] = df["Delivery Method"].astype(str)

In [187]:
for key, values in dict_Directory['Delivery Method'].items():
    df['Delivery Method'] = df['Delivery Method'].replace(values, key)

In [188]:
set(df['Delivery Method'].unique()) <= set(dict_Directory['Delivery Method'].keys())

False

In [189]:
#if not set(df["Trading Counterparty Entity"].unique()) <= set(dict_Directory['Company'].keys()):
# Nombres de company que no estan en keys
nokey_list = list(set(df['Delivery Method'].unique()) - set(dict_Directory['Delivery Method'].keys()))
nokey_list = [str(nokey) for nokey in nokey_list]

if "nan" in nokey_list:
    nokey_list.remove("nan")

nokey_list

['in-tank ',
 'In Tank Transfer',
 'by Pipeline,Book Trnsfr,Pumpovr,Barge,Vessel or Intnk Trnsfr as mutually agreed',
 'Marine Vessel',
 'Tank Transfer',
 'Marine',
 'PIPELINE',
 'Other (Railcar)',
 'In-tank',
 'Pump-Over',
 'Kinder Morgan (SF Bay Line)',
 'Head of Pipe ',
 'in-tank transfer/ pipeline',
 'Other (specify) - Bulk',
 'Pipe',
 'Other (Truck)',
 'Other (As Directed)']

In [190]:
dict_Directory['Delivery Method']['Any']

['by Pipeline or Book Trnsfr as mutually agreed',
 'by Pipeline, Pumpover, Book, In-Tank Transfer, or PTO as mutually agreed',
 'By Pipeline,Book Trnsfr,In-tank Trnsfr,or Allocation Trnsfr as mutually agreed',
 'Delivered']

In [191]:
key_list = list(dict_Directory['Delivery Method'].keys())
key_list

['Barge',
 'Book Transfer',
 'Bulk',
 'Any',
 'by Vessel or Barge',
 'In-Tank Transfer',
 'Pipeline',
 'Railcar',
 'Truck',
 'Vessel']

In [192]:
key_list = list(map(str, key_list))
key_list = [clean_string(string) for string in key_list]
key_list = [clean_string(string) for string in key_list if string is not None]
key_list

['Barge',
 'Book Transfer',
 'Bulk',
 'Any',
 'by Vessel or Barge',
 'In-Tank Transfer',
 'Pipeline',
 'Railcar',
 'Truck',
 'Vessel']

In [193]:
dictio_ = {}
for nokey in nokey_list:
    print(nokey)
    dict_ = {}
    for key in key_list:
        value_list = list(dict_Directory['Delivery Method'][key])
        dict_[key] = np.max(np.array([round(fuzz.token_set_ratio(value.upper(), nokey.upper()),2) for value in value_list]))
            
    key_max = max(dict_, key= lambda x: dict_[x])
    print(key_max)
    dictio_[nokey] = key_max

dictio_

in-tank 
Any
In Tank Transfer
In-Tank Transfer
by Pipeline,Book Trnsfr,Pumpovr,Barge,Vessel or Intnk Trnsfr as mutually agreed
Any
Marine Vessel
Vessel
Tank Transfer
In-Tank Transfer
Marine
Barge
PIPELINE
Any
Other (Railcar)
Railcar
In-tank
Any
Pump-Over
Pipeline
Kinder Morgan (SF Bay Line)
Pipeline
Head of Pipe 
Pipeline
in-tank transfer/ pipeline
Pipeline
Other (specify) - Bulk
Bulk
Pipe
Pipeline
Other (Truck)
Truck
Other (As Directed)
Pipeline


{'in-tank ': 'Any',
 'In Tank Transfer': 'In-Tank Transfer',
 'by Pipeline,Book Trnsfr,Pumpovr,Barge,Vessel or Intnk Trnsfr as mutually agreed': 'Any',
 'Marine Vessel': 'Vessel',
 'Tank Transfer': 'In-Tank Transfer',
 'Marine': 'Barge',
 'PIPELINE': 'Any',
 'Other (Railcar)': 'Railcar',
 'In-tank': 'Any',
 'Pump-Over': 'Pipeline',
 'Kinder Morgan (SF Bay Line)': 'Pipeline',
 'Head of Pipe ': 'Pipeline',
 'in-tank transfer/ pipeline': 'Pipeline',
 'Other (specify) - Bulk': 'Bulk',
 'Pipe': 'Pipeline',
 'Other (Truck)': 'Truck',
 'Other (As Directed)': 'Pipeline'}

In [194]:
# dict_Directory['Delivery Method']

In [195]:
df['Delivery Method'] = df['Delivery Method'].replace(dictio_)

In [196]:
# Nombres de company que no estan en keys
nokey_list = list(set(df['Delivery Method'].unique()) - set(dict_Directory['Delivery Method'].keys()))
nokey_list

[]

In [197]:
df['Delivery Method'] = df["Delivery Method"].map(lambda string: None if string=='nan' else string)

In [198]:
df['Delivery Method'].unique()

array(['Pipeline', 'In-Tank Transfer', 'Barge', 'Any', 'Bulk', 'Railcar',
       'Vessel', 'Truck'], dtype=object)

#### Marine Vessel Name

In [199]:
df['Marine Vessel Name'].unique()

array(['', 'n/a', '-', 'NA', 'unavailable', '(Empty)', 'N/A',
       'NOT APPLICABLE'], dtype=object)

In [200]:
df['Marine Vessel Name'] = df["Marine Vessel Name"].astype(str)

In [201]:
df['Marine Vessel Name'].unique()

array(['', 'n/a', '-', 'NA', 'unavailable', '(Empty)', 'N/A',
       'NOT APPLICABLE'], dtype=object)

In [202]:
list_nope = ['', 'N/A', 'n/a', 'NA', '(Empty)', 'nan']

In [203]:
df['Marine Vessel Name'] = df["Marine Vessel Name"].map(lambda string: None if string in list_nope else string)

In [204]:
df.loc[np.logical_and(df['Delivery Method']=="Marine Vessel", df['Marine Vessel Name'].isnull()), 'Marine Vessel Name'] = "Flag"

In [205]:
df['Marine Vessel Name'].unique()

array([None, '-', 'unavailable', 'NOT APPLICABLE'], dtype=object)

In [206]:
df['Marine Vessel Name'] = df["Marine Vessel Name"].map(lambda string: clean_string(string))

In [207]:
df['Marine Vessel Name'].unique()

array([None, 'unavailable'], dtype=object)

#### Marine Vessel IMO Number

In [208]:
df['Marine Vessel IMO Number'].unique()

array(['', 'n/a', '-', 'NA', 'unavailable', '(Empty)', 'N/A', 0.0,
       'NOT APPLICABLE'], dtype=object)

In [209]:
df['Marine Vessel IMO Number'] = df["Marine Vessel IMO Number"].astype(str)

In [210]:
df['Marine Vessel IMO Number'].unique()

array(['', 'n/a', '-', 'NA', 'unavailable', '(Empty)', 'N/A', '0.0',
       'NOT APPLICABLE'], dtype=object)

In [211]:
list_nope = ['', 'N/A', 'n/a', 'NA', '(Empty)', 'nan', '0.0']

In [212]:
df['Marine Vessel IMO Number'] = df["Marine Vessel IMO Number"].map(lambda string: None if string in list_nope else string)

In [213]:
df.loc[np.logical_and(df['Delivery Method']=="Marine Vessel", df['Marine Vessel IMO Number'].isnull()), 'Marine Vessel IMO Number'] = "Flag"

In [214]:
df['Marine Vessel IMO Number'].unique()

array([None, '-', 'unavailable', 'NOT APPLICABLE'], dtype=object)

#### Delivery Location

In [215]:
df['Delivery Location'].unique()

array(['TFMARTCA TRA', 'LOS ANGLS,CA', 'Concord CA', 'Watson CA',
       'RODEO CA PSX REN RFY ER01', 'SAN FRANCISCO AREA',
       'CONCORD CA BUPA', 'LOS ANGELES AREA',
       'WATSON, CA - KINDER MORGAN', 'Watson', 'CARSON,CA-RACK-KM',
       'CONCORD,CA-BULK-KM', 'WILMINGTON,CA-REF-VLO', 'SWT1, CA (TLC)',
       'BAY', 'Portland - KM', 'Los Angeles', 'CONCORD, CA',
       'CONCORCA GEN', 'Carson, CA (KM)', 'Carson KMEP CA',
       'Watson, CA Terminal Kinder Morgan',
       'Carson, CA Terminal Kinder Morgan', 'COLTON,CA-RACK-KM',
       'CARSON,CA-BULK-SHELL', 'LONG BEACH,CA',
       '#E101 (LOS ANGELES AREA) DEFAULT', 'Long Beach CA PDI Terminal',
       'Carson CA KM Watson Station', 'Concord CA KM PL Station',
       'Kinder Morgan Watson Station',
       'Los Angeles CA LAX Airport Terminal',
       'Barstow, CA Terminal Kinder Morgan', 'LOS ANGELES, CA',
       'Kinder Morgan LA', 'KMSF', 'KMLA w/LAX Option',
       'CARSON, CA - SOPUS', 'Portland - KMW', 'Imperial', 'Phoenix'

In [216]:
dict_Directory['Delivery Location']

{'Bakersfield': ['BAKERSFIELD CA BUPA',
  'BAKERSFIELD CA CRIMSN INV 3PTM A3CO',
  'Bakersfield Crimson Plant CA'],
 'Bay Area': ['BAY'],
 'Benecia': ['Benicia CA', 'BENICIA,CA-REF-VLO'],
 'Brisbaca': ['BRISBACA SFP'],
 'LA': ['CA LA',
  'LA AIRPORT',
  'LA Airport (LAX)',
  'Los Angeles',
  'LOS ANGELES AREA',
  'Los Angeles CA',
  'LOS ANGELES CA BUPA',
  'Los Angeles CA Trm LAX',
  'Los Angeles International Airport',
  'LOS ANGELES, CA',
  'LOS ANGLS,CA'],
 'Phoenix': ['CALJET - PHOENIX, AZ', 'Phoenix'],
 'Carson': ['CARSO-KINDER',
  'Carson CA Trm Kinder Morgan',
  'Carson KM Storage, CA',
  'Carson KMEP CA',
  'CARSON KMI',
  'Carson Olympus CA',
  'Carson, CA',
  'Carson, CA (KM)',
  'Carson, CA (Shell)',
  'CARSON,CA-BULK-KM WATSON',
  'CARSON,CA-BULK-LAX FUELS',
  'CARSON,CA-BULK-SHELL',
  'CARSON,CA-RACK-KM',
  'KINDER MORGAN - CARSON, CA'],
 'Cherry Point WA': ['Cherry Point WA'],
 'Chico': ['Chico CA Trm Kinder Morgan', 'Chico, CA', 'Chico, CA (SFPP)'],
 'Colton': ['COLTON 

In [217]:
for key, values in dict_Directory['Delivery Location'].items():
    df['Delivery Location'] = df['Delivery Location'].replace(values, key)

In [218]:
set(df['Delivery Location'].unique()) <= set(dict_Directory['Delivery Location'].keys())

False

In [219]:
#if not set(df["Trading Counterparty Entity"].unique()) <= set(dict_Directory['Company'].keys()):
# Nombres de company que no estan en keys
nokey_list = list(set(df['Delivery Location'].unique()) - set(dict_Directory['Delivery Location'].keys()))
nokey_list = [str(nokey) for nokey in nokey_list]
nokey_list = [clean_string(string) for string in nokey_list]
nokey_list = [string for string in nokey_list if string is not None]
nokey_list

['Watson, CA Terminal Kinder Morgan',
 'SAN JOSE, CA - KINDER MORGAN',
 'Richmond, CA 3rd Party',
 'SACRAMENTO, CA - KINDER MORGAN',
 'Carson CA KM GATX Terminal',
 'Portland-McCall',
 'Los Angeles CA LAX Airport Terminal',
 'Reno, NV',
 'Concord, CA-Bulk',
 'Richmond CA Rfy Chevron',
 'Long Beach CA PDI Terminal',
 'CONCORD, CA',
 'Watson, CA-Bulk',
 'CONCORD CA BUPA',
 'KMSF',
 'Long Beach, CA (Olympus)',
 'Kinder Morgan Watson Station',
 'LAXFUEL',
 'Richmond CA Chevron Long Wharf',
 'Barstow, CA Terminal Kinder Morgan',
 'Phoenix, AZ',
 'SAN FRANCISCO AIRPORT SFO, CA',
 'Phoenix, AZ (CalJet)',
 'Imperial KMEP CA',
 'CONCORD, CA - KM PUMPSTATION',
 'Watson, CA Trm KiM or LAX',
 'Las Vegas, NV',
 'RODEO CA PSX REN RFY ER01',
 'Portland - KMW',
 'Phoenix, AZ (SFPP)',
 'Kinder Morgan LA',
 'GATX',
 'OAKLAND INTL. AIRPORT OAK, CA',
 'Carson CA KM GATX/SoCal PL Junction',
 'Long Beach CA Olympus Terminal',
 'WATSON KMO',
 'Colton, CA Terminal Kinder Morgan',
 'Carson CA KM Watson Station

In [220]:
key_list = list(dict_Directory['Delivery Location'].keys())
key_list = list(map(str, key_list))
key_list = [clean_string(string) for string in key_list]
key_list = [string for string in key_list if string is not None]

In [221]:
def max_keys(d):
    max_value = max(d.values())
    return [key for key, value in d.items() if value == max_value]

In [222]:
dictio_ = {}
for nokey in nokey_list:
    # print(nokey)
    dict_ = {}
    for key in key_list:
        # print(key)
        value_list = list(dict_Directory['Delivery Location'][key])
        dict_[key] = np.max(np.array([round(fuzz.token_set_ratio(value.upper(), nokey.upper()),2) for value in value_list]))
            
    #key_max = max(dict_, key= lambda x: dict_[x])
    key_max = max_keys(dict_)
    value_max = max(dict_.values())

    dictio_[nokey] = key_max
    
    if len(key_max)>1:
        # 'LA Area': ['LA', 'San Francisco']  ----> 'LA Area': ['LA']
        if 'LA' in key_max and 'LA' in nokey.split():
            dictio_[nokey] = ['LA']
            value_max = 90

    elif len(key_max)==1:
        for k in [k for k in list(dict_Directory['Delivery Location'].keys()) if len(str(k).split())==1]:
            if str(k).upper() in [str(x).upper() for x in nokey.replace(',', '').split()]:
                dictio_[nokey] = [k]
                value_max = 90

    if value_max < 90:
        dictio_[nokey] = ['none']  
        
#    print(nokey)
#    print(key_max)
#    print(value_max)

In [223]:
dictio_

{'Watson, CA Terminal Kinder Morgan': ['Watson'],
 'SAN JOSE, CA - KINDER MORGAN': ['none'],
 'Richmond, CA 3rd Party': ['Richmond'],
 'SACRAMENTO, CA - KINDER MORGAN': ['none'],
 'Carson CA KM GATX Terminal': ['Carson'],
 'Portland-McCall': ['none'],
 'Los Angeles CA LAX Airport Terminal': ['LA'],
 'Reno, NV': ['Reno'],
 'Concord, CA-Bulk': ['Concord'],
 'Richmond CA Rfy Chevron': ['Richmond'],
 'Long Beach CA PDI Terminal': ['Long Beach'],
 'CONCORD, CA': ['Concord'],
 'Watson, CA-Bulk': ['Watson'],
 'CONCORD CA BUPA': ['Concord'],
 'KMSF': ['none'],
 'Long Beach, CA (Olympus)': ['Long Beach'],
 'Kinder Morgan Watson Station': ['Watson'],
 'LAXFUEL': ['none'],
 'Richmond CA Chevron Long Wharf': ['Richmond'],
 'Barstow, CA Terminal Kinder Morgan': ['none'],
 'Phoenix, AZ': ['Phoenix', 'Watson'],
 'SAN FRANCISCO AIRPORT SFO, CA': ['none'],
 'Phoenix, AZ (CalJet)': ['Phoenix'],
 'Imperial KMEP CA': ['Imperial'],
 'CONCORD, CA - KM PUMPSTATION': ['Concord'],
 'Watson, CA Trm KiM or LAX':

In [224]:
sum([len(x) for x in list(dictio_.values())])

55

In [225]:
len([len(x) for x in list(dictio_.values())])

54

In [226]:
# if sum([len(x) for x in list(dictio_.values())]) == len([len(x) for x in list(dictio_.values())]):
dict_ = {}
for key, value in dictio_.items():
    dict_[key] = value[0]

In [227]:
df['Delivery Location'] = df['Delivery Location'].replace(dict_)

In [228]:
df['Delivery Location'] = df["Delivery Location"].map(lambda string: clean_string(string))

In [229]:
df['Delivery Location'].unique()

array(['TFMARTCA', 'LA', 'Concord', 'Watson', None, 'San Francisco',
       'Carson', 'Wilmington', 'SWT', 'Bay Area', 'Portland', 'Colton',
       'Long Beach', 'Imperial', 'Phoenix', 'Richmond', 'Las Vegas',
       'Reno'], dtype=object)

#### Pipeline Delivery Sub-cycle

In [230]:
df['Pipeline Delivery Sub-cycle'] = df["Pipeline Delivery Sub-cycle"].astype(str)

In [231]:
df['Pipeline Delivery Sub-cycle'].unique()

array(['KM North FH', 'KM West Any', '', 'KM Any', 'n/a', 'PLVW-X6-002',
       'Any', 'ANY', 'Other (1/17, 1/22, 1/27)', 'Prompt', 'any',
       'Other (Jan 18-19)', 'C2', '-', 'Other (1/20-1/31)', 'PLVW-X6-004',
       'SHLW-X6-004', 'VALW-H5-302', '02-C1-2025', '02-C2-2025',
       '02-C3-2025', '02-C4-2025', '01-C3-2025',
       'Other (Januray 2025 - ANY)', 'Other (1/16, 1/17 Loadings)',
       'Other (March 2025 - ~165 b per cycle)',
       'Other (February 2025 - ~112.5mb per cycle)', 'C3', 'C1', 'C4',
       'KMW 2025 - Feb Any', '112.5MB/ Cy', '165 MB/ Cy', 'KMW 2025 - 03',
       'KM West C4', 'KM West C3', 'Other (Specify) C3-4', 'Jan',
       'Other (Q1 2025 - 25 Jan, 25 Feb, 25 Mar)', '(Empty)',
       'KMR 2025 - 03', 'N/A', 'KM North C3',
       'Other (February 2025 - ANY)', 'Other (ANY Jan)',
       'KMW 2025 - Jan Any', 'KMR 2025 - 04',
       'Other (January 2025 - ANY)', 'Other (March 2025 - ANY)',
       'Other (2/1-2/10)', 'KM South L3', 'Rateable',
       'Other 

#### Delivery Start Date

In [232]:
df['Delivery Start Date'] = df["Delivery Start Date"].astype(str)

In [233]:
df['Delivery Start Date'].unique()

array(['2025-01-15 06:00:00', '2025-02-01 06:00:00',
       '2025-03-01 06:00:00', '2025-01-01', '2025-01-03', '01/01/2025',
       '01/03/2025', '2025-01-17', '2025-02-01', '2025-01-10',
       '2025-01-16', '2025-01-25', '2025-01-06', '2025-01-18',
       '02/01/2025', '01/06/2025', '01/09/2025', '2025-01-07 06:00:00',
       '2025-01-20', '2025-03-01', '2025-01-07', 'Jan 2025', 'Feb 2025',
       '2025-06-01', '2025-05-01', '2025-04-01', '2025-01-08',
       '2025-01-22', '', '2025-01-09', '2025-01-23', '2024-12-01',
       '2025-01-19', '2025-01-14', '2025-01-13', '2025-01-13 06:00:00',
       '01/13/2025', '01/18/2025', '2025-01-14 06:00:00',
       '2025-01-22 06:00:00', '2025-02-08 06:00:00',
       '2025-02-15 06:00:00', '2025-02-22 06:00:00', '01/15/2025',
       '01/14/2025', '2025-01-12', '2025-01-15', 'Mar 2025', '2025-03-12',
       '03/01/2025', '2024-08-06', '2024-08-13', '2024-07-28',
       '2024-07-29', '2024-08-21', '2024-08-28', '2024-09-05',
       '2024-09-13', '2

In [234]:
df['Delivery Start Date'] = df["Delivery Start Date"].map(lambda x: str_to_date(x))

In [235]:
df['Delivery Start Date'].unique()

array([datetime.date(2025, 1, 15), datetime.date(2025, 2, 1),
       datetime.date(2025, 3, 1), datetime.date(2025, 1, 1),
       datetime.date(2025, 1, 3), datetime.date(2025, 1, 17),
       datetime.date(2025, 1, 10), datetime.date(2025, 1, 16),
       datetime.date(2025, 1, 25), datetime.date(2025, 1, 6),
       datetime.date(2025, 1, 18), datetime.date(2025, 1, 9),
       datetime.date(2025, 1, 7), datetime.date(2025, 1, 20), NaT,
       datetime.date(2025, 6, 1), datetime.date(2025, 5, 1),
       datetime.date(2025, 4, 1), datetime.date(2025, 1, 8),
       datetime.date(2025, 1, 22), datetime.date(2025, 1, 23),
       datetime.date(2024, 12, 1), datetime.date(2025, 1, 19),
       datetime.date(2025, 1, 14), datetime.date(2025, 1, 13),
       datetime.date(2025, 2, 8), datetime.date(2025, 2, 15),
       datetime.date(2025, 2, 22), datetime.date(2025, 1, 12),
       datetime.date(2025, 3, 12), datetime.date(2024, 8, 6),
       datetime.date(2024, 8, 13), datetime.date(2024, 7, 28),


In [236]:
df['Delivery Start Date'].unique()

array([datetime.date(2025, 1, 15), datetime.date(2025, 2, 1),
       datetime.date(2025, 3, 1), datetime.date(2025, 1, 1),
       datetime.date(2025, 1, 3), datetime.date(2025, 1, 17),
       datetime.date(2025, 1, 10), datetime.date(2025, 1, 16),
       datetime.date(2025, 1, 25), datetime.date(2025, 1, 6),
       datetime.date(2025, 1, 18), datetime.date(2025, 1, 9),
       datetime.date(2025, 1, 7), datetime.date(2025, 1, 20), NaT,
       datetime.date(2025, 6, 1), datetime.date(2025, 5, 1),
       datetime.date(2025, 4, 1), datetime.date(2025, 1, 8),
       datetime.date(2025, 1, 22), datetime.date(2025, 1, 23),
       datetime.date(2024, 12, 1), datetime.date(2025, 1, 19),
       datetime.date(2025, 1, 14), datetime.date(2025, 1, 13),
       datetime.date(2025, 2, 8), datetime.date(2025, 2, 15),
       datetime.date(2025, 2, 22), datetime.date(2025, 1, 12),
       datetime.date(2025, 3, 12), datetime.date(2024, 8, 6),
       datetime.date(2024, 8, 13), datetime.date(2024, 7, 28),


#### Delivery End Date

In [237]:
df['Delivery End Date'] = df["Delivery End Date"].astype(str)

In [238]:
df['Delivery End Date'].unique()

array(['2025-01-22 06:00:00', '2025-02-07 06:00:00',
       '2025-03-14 05:00:00', '2025-01-31', '2025-01-07', '01/31/2025',
       '01/03/2025', '2025-02-28', '2025-01-17', '2025-01-25',
       '2025-01-19', '02/28/2025', '01/07/2025', '01/10/2025',
       '2025-02-07', '2025-01-31 06:00:00', '2025-03-31', 'Jan 2025',
       'Feb 2025', '2025-06-30', '2025-05-31', '2025-04-30', '2025-01-18',
       '2025-01-23', '', '2025-01-24', '2025-01-20', '2024-12-31',
       '2025-01-21 06:00:00', '2025-01-21', '2025-01-14', '2025-01-16',
       '2025-02-28 06:00:00', '01/23/2025', '2025-02-10',
       '2025-02-14 06:00:00', '2025-02-21 06:00:00', '01/15/2025',
       '2025-03-07', '2025-01-02', '2025-01-11', '2025-01-12',
       '2025-01-15', 'Mar 2025', '2025-03-22', '03/31/2025',
       '2025-02-15 06:00:00', '2024-08-06', '2024-08-13', '2024-08-02',
       '2024-07-30', '2024-08-21', '2024-08-28', '2024-09-05',
       '2024-09-13', '2024-09-20', '2024-10-06', '2024-10-13',
       '2024-10-21

In [239]:
df['Delivery End Date'] = df["Delivery End Date"].map(lambda x: str_to_date(x))

In [240]:
df['Delivery End Date'].unique()

array([datetime.date(2025, 1, 22), datetime.date(2025, 2, 7),
       datetime.date(2025, 3, 14), datetime.date(2025, 1, 31),
       datetime.date(2025, 1, 7), datetime.date(2025, 1, 3),
       datetime.date(2025, 2, 28), datetime.date(2025, 1, 17),
       datetime.date(2025, 1, 25), datetime.date(2025, 1, 19),
       datetime.date(2025, 1, 10), datetime.date(2025, 3, 31), NaT,
       datetime.date(2025, 6, 30), datetime.date(2025, 5, 31),
       datetime.date(2025, 4, 30), datetime.date(2025, 1, 18),
       datetime.date(2025, 1, 23), datetime.date(2025, 1, 24),
       datetime.date(2025, 1, 20), datetime.date(2024, 12, 31),
       datetime.date(2025, 1, 21), datetime.date(2025, 1, 14),
       datetime.date(2025, 1, 16), datetime.date(2025, 2, 10),
       datetime.date(2025, 2, 14), datetime.date(2025, 2, 21),
       datetime.date(2025, 1, 15), datetime.date(2025, 3, 7),
       datetime.date(2025, 1, 2), datetime.date(2025, 1, 11),
       datetime.date(2025, 1, 12), datetime.date(2025,

In [241]:
# def date_to_str(date):
#     if pd.isnull(date): return None
#     else: return date.strftime("%Y-%m-%d")

In [242]:
# df['Delivery End Date'] = df["Delivery End Date"].map(lambda date: date_to_str(date))

In [243]:
df['Delivery End Date'].unique()

array([datetime.date(2025, 1, 22), datetime.date(2025, 2, 7),
       datetime.date(2025, 3, 14), datetime.date(2025, 1, 31),
       datetime.date(2025, 1, 7), datetime.date(2025, 1, 3),
       datetime.date(2025, 2, 28), datetime.date(2025, 1, 17),
       datetime.date(2025, 1, 25), datetime.date(2025, 1, 19),
       datetime.date(2025, 1, 10), datetime.date(2025, 3, 31), NaT,
       datetime.date(2025, 6, 30), datetime.date(2025, 5, 31),
       datetime.date(2025, 4, 30), datetime.date(2025, 1, 18),
       datetime.date(2025, 1, 23), datetime.date(2025, 1, 24),
       datetime.date(2025, 1, 20), datetime.date(2024, 12, 31),
       datetime.date(2025, 1, 21), datetime.date(2025, 1, 14),
       datetime.date(2025, 1, 16), datetime.date(2025, 2, 10),
       datetime.date(2025, 2, 14), datetime.date(2025, 2, 21),
       datetime.date(2025, 1, 15), datetime.date(2025, 3, 7),
       datetime.date(2025, 1, 2), datetime.date(2025, 1, 11),
       datetime.date(2025, 1, 12), datetime.date(2025,

#### Price Basis Method

In [244]:
df['Price Basis Method_'] = df["Price Basis Method"].copy()

In [245]:
df['Price Basis Method'] = df["Price Basis Method"].astype(str)
df['Price Basis Method'].unique()

array(['Floating Event', 'Floating', 'EFP', 'Index', 'Dates 111',
       'Floating - Event', 'Fixed', 'Other (API_Index Sale with Diff)',
       'OPIS 3 Day', 'Floating Deemed', 'Efp', 'Other (Index Sale)',
       'Floating - Deemed', 'MPC Formula Tablet',
       'MPC EFP Floating Price', 'Deemed Date', 'Fixed Price',
       'Index Price with Diff', 'Other (Index Purchase)',
       'Floating-Floating', 'OPIS Print', 'Other (Index Sale with Diff)',
       'Deemed', 'FLOATING', 'Dates 212', 'TAS', 'Deemed-Floating',
       'Dates 003', 'INDEX',
       'OPIS LA +0 Index on 3 day wrap from pump date'], dtype=object)

In [246]:
df['Price Basis Method'] = df["Price Basis Method"].map(lambda string: clean_string(string))
df['Price Basis Method'].unique()

array(['Floating Event', 'Floating', 'EFP', 'Index', 'Dates 111',
       'Floating - Event', 'Fixed', 'Other (API_Index Sale with Diff)',
       'OPIS 3 Day', 'Floating Deemed', 'Efp', 'Other (Index Sale)',
       'Floating - Deemed', 'MPC Formula Tablet',
       'MPC EFP Floating Price', 'Deemed Date', 'Fixed Price',
       'Index Price with Diff', 'Other (Index Purchase)',
       'Floating-Floating', 'OPIS Print', 'Other (Index Sale with Diff)',
       'Deemed', 'FLOATING', 'Dates 212', 'TAS', 'Deemed-Floating',
       'Dates 003', 'INDEX',
       'OPIS LA +0 Index on 3 day wrap from pump date'], dtype=object)

In [247]:
dict_Directory.keys()

dict_keys(['Company', 'Position Seq. Num.', 'CPID', 'Counterparty Contact', 'TL', 'Sell_Buy', 'Product Type', 'Product', 'Delivery Method', 'Delivery Location', 'Price Basis Method', 'Reference Product'])

In [248]:
dict_Directory['Price Basis Method'].keys()

dict_keys(['Deemed', 'EFO', 'EFP', 'Fixed', 'Floating-Event', 'Floating-Price', 'Not A Trade', 'OPIS', 'Platts'])

In [249]:
dictio = dict_Directory['Price Basis Method']
df_ = pd.DataFrame(df['Price Basis Method'].unique())
df_.columns = ['Price Basis Method']
df_['KEY CS=True'] = df_["Price Basis Method"].map(lambda string: match_dictio(string, dictio, case_sensitive=True))
df_['KEY CS=False'] = df_["Price Basis Method"].map(lambda string: match_dictio(string, dictio, case_sensitive=False))
df_

,Price Basis Method,KEY CS=True,KEY CS=False
0,Floating Event,None,Floating-Event
1,Floating,Floating-Event,Floating-Event
2,EFP,EFP,EFP
3,Index,None,EFP
4,Dates 111,Floating-Event,Floating-Event
5,Floating - Event,Floating-Event,Floating-Event
6,Fixed,Fixed,Fixed
7,Other (API_Index Sale with Diff),EFP,EFP
8,OPIS 3 Day,OPIS,OPIS
9,Floating Deemed,None,Deemed


In [250]:
dictio_ = {}
# dictio_max = {}
for nokey in df_['Price Basis Method']:
    dict_ = {}
    for key in list(dict_Directory['Price Basis Method'].keys()):
        value_list = list(dict_Directory['Price Basis Method'][key]) + [key]
        value_list = [str(value).strip() for value in value_list]
        # dict_[key] = np.max(np.array([round(fuzz.token_set_ratio(value.upper(), nokey.upper()),2) for value in value_list]))
        # dict_[key] = np.max(np.array([round(fuzz.ratio(value.upper(), nokey.upper()),2) for value in value_list]))
        # dict_[key] = np.max(np.array([round(fuzz.partial_ratio(value.upper(), nokey.upper()),2) for value in value_list]))
        dict_[key] = np.max(np.array([round(jaro_similarity(str(value).upper(), str(nokey).upper()),2) for value in value_list]))*100
        
    # key_max = max(dict_, key= lambda x: dict_[x])
    # dictio_[nokey] = key_max
    # dictio_max[nokey] = max(dict_.values())
    if max(dict_.values())>=70:
        dictio_[nokey] = [max(dict_, key= lambda x: dict_[x]), max(dict_.values())]

In [251]:
dictio_

{'Floating Event': ['Floating-Event', 100.0],
 'Floating': ['Floating-Event', 100.0],
 'EFP': ['EFP', 100.0],
 'Index': ['EFP', 100.0],
 'Dates 111': ['Floating-Event', 100.0],
 'Floating - Event': ['Floating-Event', 100.0],
 'Fixed': ['Fixed', 100.0],
 'Other (API_Index Sale with Diff)': ['EFP', 100.0],
 'OPIS 3 Day': ['OPIS', 100.0],
 'Floating Deemed': ['Deemed', 100.0],
 'Efp': ['EFP', 100.0],
 'Other (Index Sale)': ['EFP', 100.0],
 'Floating - Deemed': ['Deemed', 100.0],
 'MPC Formula Tablet': ['Fixed', 100.0],
 'MPC EFP Floating Price': ['Floating-Event', 100.0],
 'Deemed Date': ['Deemed', 100.0],
 'Fixed Price': ['Fixed', 100.0],
 'Index Price with Diff': ['EFP', 100.0],
 'Other (Index Purchase)': ['EFP', 100.0],
 'Floating-Floating': ['Floating-Event', 82.0],
 'OPIS Print': ['OPIS', 100.0],
 'Other (Index Sale with Diff)': ['EFP', 100.0],
 'Deemed': ['Deemed', 100.0],
 'FLOATING': ['Floating-Event', 100.0],
 'Dates 212': ['Floating-Event', 100.0],
 'TAS': ['EFP', 100.0],
 'Deem

In [252]:
dict_ = {}
for key, value in dictio_.items():
    dict_[key] = value[0]

df_['KEY Fuzzy'] = df_['Price Basis Method'].map(dict_).fillna("")
df_['KEY Fuzzy'] = df_["KEY Fuzzy"].map(lambda string: clean_string(string))
df_

,Price Basis Method,KEY CS=True,KEY CS=False,KEY Fuzzy
0,Floating Event,None,Floating-Event,Floating-Event
1,Floating,Floating-Event,Floating-Event,Floating-Event
2,EFP,EFP,EFP,EFP
3,Index,None,EFP,EFP
4,Dates 111,Floating-Event,Floating-Event,Floating-Event
5,Floating - Event,Floating-Event,Floating-Event,Floating-Event
6,Fixed,Fixed,Fixed,Fixed
7,Other (API_Index Sale with Diff),EFP,EFP,EFP
8,OPIS 3 Day,OPIS,OPIS,OPIS
9,Floating Deemed,None,Deemed,Deemed


In [253]:
dictio_true = dict(zip(df_['Price Basis Method'], df_['KEY CS=True']))
dictio_true = {k: v for k, v in dictio_true.items() if v is not None}
dictio_true

{'Floating': 'Floating-Event',
 'EFP': 'EFP',
 'Dates 111': 'Floating-Event',
 'Floating - Event': 'Floating-Event',
 'Fixed': 'Fixed',
 'Other (API_Index Sale with Diff)': 'EFP',
 'OPIS 3 Day': 'OPIS',
 'Other (Index Sale)': 'EFP',
 'Floating - Deemed': 'Deemed',
 'MPC Formula Tablet': 'Fixed',
 'MPC EFP Floating Price': 'Floating-Event',
 'Deemed Date': 'Deemed',
 'Fixed Price': 'Fixed',
 'Index Price with Diff': 'EFP',
 'Other (Index Purchase)': 'EFP',
 'OPIS Print': 'OPIS',
 'Other (Index Sale with Diff)': 'EFP',
 'Deemed': 'Deemed',
 'Dates 212': 'Floating-Event',
 'TAS': 'EFP',
 'Deemed-Floating': 'Deemed',
 'Dates 003': 'Floating-Event',
 'INDEX': 'EFP',
 'OPIS LA +0 Index on 3 day wrap from pump date': 'OPIS'}

In [254]:
dictio_false = dict(zip(df_['Price Basis Method'], df_['KEY CS=False']))
dictio_false = {k: v for k, v in dictio_false.items() if v is not None}
dictio_false

{'Floating Event': 'Floating-Event',
 'Floating': 'Floating-Event',
 'EFP': 'EFP',
 'Index': 'EFP',
 'Dates 111': 'Floating-Event',
 'Floating - Event': 'Floating-Event',
 'Fixed': 'Fixed',
 'Other (API_Index Sale with Diff)': 'EFP',
 'OPIS 3 Day': 'OPIS',
 'Floating Deemed': 'Deemed',
 'Efp': 'EFP',
 'Other (Index Sale)': 'EFP',
 'Floating - Deemed': 'Deemed',
 'MPC Formula Tablet': 'Fixed',
 'MPC EFP Floating Price': 'Floating-Event',
 'Deemed Date': 'Deemed',
 'Fixed Price': 'Fixed',
 'Index Price with Diff': 'EFP',
 'Other (Index Purchase)': 'EFP',
 'OPIS Print': 'OPIS',
 'Other (Index Sale with Diff)': 'EFP',
 'Deemed': 'Deemed',
 'FLOATING': 'Floating-Event',
 'Dates 212': 'Floating-Event',
 'TAS': 'EFP',
 'Deemed-Floating': 'Deemed',
 'Dates 003': 'Floating-Event',
 'INDEX': 'EFP',
 'OPIS LA +0 Index on 3 day wrap from pump date': 'OPIS'}

In [255]:
dictio_Fuzzy = dict(zip(df_['Price Basis Method'], df_['KEY Fuzzy']))
dictio_Fuzzy = {k: v for k, v in dictio_Fuzzy.items() if v is not None}
dictio_Fuzzy

{'Floating Event': 'Floating-Event',
 'Floating': 'Floating-Event',
 'EFP': 'EFP',
 'Index': 'EFP',
 'Dates 111': 'Floating-Event',
 'Floating - Event': 'Floating-Event',
 'Fixed': 'Fixed',
 'Other (API_Index Sale with Diff)': 'EFP',
 'OPIS 3 Day': 'OPIS',
 'Floating Deemed': 'Deemed',
 'Efp': 'EFP',
 'Other (Index Sale)': 'EFP',
 'Floating - Deemed': 'Deemed',
 'MPC Formula Tablet': 'Fixed',
 'MPC EFP Floating Price': 'Floating-Event',
 'Deemed Date': 'Deemed',
 'Fixed Price': 'Fixed',
 'Index Price with Diff': 'EFP',
 'Other (Index Purchase)': 'EFP',
 'Floating-Floating': 'Floating-Event',
 'OPIS Print': 'OPIS',
 'Other (Index Sale with Diff)': 'EFP',
 'Deemed': 'Deemed',
 'FLOATING': 'Floating-Event',
 'Dates 212': 'Floating-Event',
 'TAS': 'EFP',
 'Deemed-Floating': 'Deemed',
 'Dates 003': 'Floating-Event',
 'INDEX': 'EFP',
 'OPIS LA +0 Index on 3 day wrap from pump date': 'OPIS'}

In [256]:
dictio = dict(dictio_Fuzzy)

In [257]:
for k, v in dictio.items():
    if k in list(dictio_false.keys()):
        dictio[k] = dictio_false[k]
    if k in list(dictio_true.keys()):
        dictio[k] = dictio_true[k]

In [258]:
df['Price Basis Method'] = df['Price Basis Method'].replace(dictio)

In [259]:
(set(df['Price Basis Method'].unique()) - {None}) <= set(dict_Directory['Price Basis Method'].keys())

True

In [260]:
(set(df['Price Basis Method'].unique()) - {None}) - set(dict_Directory['Price Basis Method'].keys())

set()

#### Reference Product

In [261]:
df['Reference Product'] = df["Reference Product"].astype(str)

In [262]:
df['Reference Product'] = df["Reference Product"].map(lambda string: clean_string(string))

In [263]:
df['Reference Product'].unique()

array(['100% Opis.Suboctane.San Francisc.Spot.M -0.5 Usc/Usg',
       '100% Opis.Carbob.Pipeline.Los Angeles.Spot.M', 'OPIS-Sub Octane',
       'NYMEX RBOB-R', 'Multiple Multiple Multiple',
       'OPIS Spot CARBOB REG BAY AREA',
       'NYMEX RB (NYMX) - RBOB GASOLINE FUTURES N/A', 'OPS_LA_AZ_CBG_UNL',
       'NYMEX RBOB',
       '(1)*OP LA CARB NO2-C + (1)*OP CA CARBON CPG DSL LCFS/CRUDE CI COMBINED-C + (1)*OP CA CAR CARB NO2-C',
       'NYMEX RBOB Gasoline', None, 'OPIS WC Spot +  ECA Marine Diesel',
       'CARBOB Regular', 'Conventional Regular', 'NYMEX ULSD',
       'NYMEX HO (NYMX) - NY HARBOR ULSD FUTURES N/A',
       '100% Opis.Carbob.Pipeline.San Francisc.Spot.M',
       '100% Opis.Ulsd.Pipeline.San Francisc.Spot.M +0 Usc/Usg',
       'Nymex Ho', 'NYMEX HO', 'OPIS WC Spot +  No 2 CARB ULSD 15',
       'OPIS CARB No2', 'NYMEX - RBOB',
       'CARB Diesel @ USWC LA, CA Carbon Dsl Combi LCFS/Crude @ CA, US Bio RIN @ USA, & CAR Carb Diesel @ USA',
       '(1)*OP SF CARBOB UNL-M',

In [264]:
dict_Directory['Reference Product'].keys()

dict_keys(['NYMEX ULSD', 'Platts ULSD', 'OPIS ULS No2', 'OPIS RBOB-R', 'OPIS AZRBOB-R', 'OPIS CARB No2', 'OPIS CARBOB-P', 'OPIS CARBOB-R', 'OPIS JET-A', 'OPIS CBOB-P', 'OPIS CBOB-R', '', 'NYMEX RBOB', 'OPIS AZRBOB-P', 'OPIS ETHANOL'])

In [265]:
dictio = dict_Directory['Reference Product']
df_ = df['Reference Product'].unique()
df_ = pd.DataFrame(df['Reference Product'].unique())
df_.columns = ['Reference Product']
df_['Reference Product KEY CS=True'] = df_["Reference Product"].map(lambda string: match_dictio(string, dictio, case_sensitive=True))
df_['Reference Product KEY CS=False'] = df_["Reference Product"].map(lambda string: match_dictio(string, dictio, case_sensitive=False))
df_.head(4)

,Reference Product,Reference Product KEY CS=True,Reference Product KEY CS=False
0,100% Opis.Suboctane.San Francisc.Spot.M -0.5 U...,None,OPIS CBOB-R
1,100% Opis.Carbob.Pipeline.Los Angeles.Spot.M,None,OPIS CARBOB-R
2,OPIS-Sub Octane,None,None
3,NYMEX RBOB-R,NYMEX RBOB,NYMEX RBOB


In [266]:
print('Total:', df_.shape[0])
print('Total w.o None:', df_[~df_['Reference Product'].isnull()].shape[0])
print('Reference Product KEY CS=True:', df_[~df_['Reference Product KEY CS=True'].isnull()].shape[0])
print('Reference Product KEY CS=False:', df_[~df_['Reference Product KEY CS=False'].isnull()].shape[0])

Total: 120
Total w.o None: 119
Reference Product KEY CS=True: 61
Reference Product KEY CS=False: 70


In [267]:
print('Total:', df.shape[0])
print('Total w.o None:', df[~df['Reference Product'].isnull()].shape[0])

Total: 663
Total w.o None: 617


In [268]:
dictio_true = dict(zip(df_['Reference Product'], df_['Reference Product KEY CS=True']))
for key, value in dictio_true.items():
    # print(key, value)
    df['Reference Product'] = df['Reference Product'].replace(str(key), str(value))

In [269]:
df['Reference Product'] = df["Reference Product"].map(lambda string: clean_string(string))

In [270]:
dictio_false = dict(zip(df_['Reference Product'], df_['Reference Product KEY CS=False']))
for key, value in dictio_false.items():
    if dictio_true[key] is None:
        df['Reference Product'] = df['Reference Product'].replace(str(key), str(value))

In [271]:
df['Reference Product'] = df["Reference Product"].map(lambda string: clean_string(string))

In [272]:
df['Reference Product'].unique()

array([None, 'NYMEX RBOB', 'OPIS CARBOB-R', 'OPIS CBOB-R', 'NYMEX ULSD',
       'OPIS CARB No2', 'OPIS ULS No2', 'OPIS CARBOB-P', 'OPIS AZRBOB-R',
       'OPIS JET-A', 'OPIS CBOB-P', 'OPIS RBOB-R', 'OPIS AZRBOB-P'],
      dtype=object)

In [273]:
df['Reference Product'] = df["Reference Product"].map(lambda string: clean_string(string))

In [274]:
df['Reference Product'].unique()

array([None, 'NYMEX RBOB', 'OPIS CARBOB-R', 'OPIS CBOB-R', 'NYMEX ULSD',
       'OPIS CARB No2', 'OPIS ULS No2', 'OPIS CARBOB-P', 'OPIS AZRBOB-R',
       'OPIS JET-A', 'OPIS CBOB-P', 'OPIS RBOB-R', 'OPIS AZRBOB-P'],
      dtype=object)

In [275]:
(set(df['Reference Product'].unique()) - {None}) <= set(dict_Directory['Reference Product'].keys())

True

#### Reference Month

In [276]:
df['Reference Month'] = df["Reference Month"].astype(str)

In [277]:
df['Reference Month'].unique()

array(['', '2025-02-01', '(blank)', '2025-01', 'February', 'FEB-25',
       '(Empty)', 'Jan', 'Feb', 'February-2025', '2025-02-25',
       '2025-01-20', '2025-03-01', '2025-02', '1900-01-01', 'MAR-25',
       'Mar', 'March', '2025-07-01', '2025-06-01', '2025-05-01',
       '2025-04-01', '2025-07-25', '2025-06-25', '2025-05-25', ' ',
       '2025-03-25', '2025-04-25', 'FEB', 'Dec', '#Err', 'JAN ',
       '2025-03', '2025-01-25', 'Apr', 'N/A', 'NOT APPLICABLE', 'NA'],
      dtype=object)

In [278]:
df['Reference Month'] = df["Reference Month"].map(lambda string: clean_string(string))

In [279]:
df['Reference Month'].unique()

array([None, '2025-02-01', '2025-01', 'February', 'FEB-25', 'Jan', 'Feb',
       'February-2025', '2025-02-25', '2025-01-20', '2025-03-01',
       '2025-02', '1900-01-01', 'MAR-25', 'Mar', 'March', '2025-07-01',
       '2025-06-01', '2025-05-01', '2025-04-01', '2025-07-25',
       '2025-06-25', '2025-05-25', '2025-03-25', '2025-04-25', 'FEB',
       'Dec', 'JAN ', '2025-03', '2025-01-25', 'Apr'], dtype=object)

In [280]:
# for string in df['Reference Month']:
#     print(string)
#     str_to_month(string)

In [281]:
df['Reference Month'] = df["Reference Month"].map(lambda string: str_to_month(string))

In [282]:
df['Reference Month'].unique()

array([None, 'Feb-2025', 'Jan-2025', 'Mar-2025', 'Jan-1900', 'Jul-2025',
       'Jun-2025', 'May-2025', 'Apr-2025', 'Dec-2025'], dtype=object)

#### Price Differential vs. Reference

In [283]:
df['Price Differential vs. Reference'] = df['Price Differential vs. Reference'].astype(str)

In [284]:
df['Price Differential vs. Reference'] = df["Price Differential vs. Reference"].map(lambda x: extract_numbers(x))
df['Price Differential vs. Reference'] = df['Price Differential vs. Reference'].map(lambda string: str_to_float(string))

In [285]:
df['Price Differential vs. Reference'] = np.where(np.logical_and(df['Product Type'].isin(['Gasoline', 'Diesel']), df['Price Differential vs. Reference']<1, df['Price Differential vs. Reference']>-1), 
                                                                     df['Price Differential vs. Reference']*100, 
                                                                     df['Price Differential vs. Reference'])

In [286]:
df['Price Differential vs. Reference'].unique()

array([ -50.    ,       nan,   11.    ,  -35.    ,    0.    ,   11.5   ,
         -0.15  ,   10.    ,    7.5   ,   -4.    ,    8.5   ,    4.    ,
          6.    ,    5.5   ,   25.    ,   26.    ,  -14.    ,   -0.14  ,
          2.    ,    1.    ,    1.25  ,    6.5   ,    3.    ,  -15.    ,
          2.7   ,   -9.5   ,   24.    , -400.    , -200.    ,   16.    ,
          8.    , -100.    , -300.    ,   -3.    ,    4.5   ,   -1.5   ,
          9.    ,  -13.    ,   -2.5   , -450.    ,    5.    ,  -11.    ,
          4.06  ,  -11.5   ,   14.    ,    3.25  ,   50.    ,   -0.5   ,
         28.    ,    0.75  ,    7.    ,   28.    ,   27.    ,   -1.    ,
         12.    ,  -75.    ,    7.    ,    0.01  ,   12.5   ,   13.5   ,
         20.    , -125.    ,  -25.    ,   13.    , -150.    ,    3.5   ,
         15.    ,   19.    ,    0.0185,   17.    ,   95.    ,   21.    ,
          1.85  , -250.    ,   -0.25  ,    3.5   ,  -18.    ,  -10.    ,
         -0.04  ,   -7.    ])

#### Pricing Event Type

In [287]:
df["Pricing Event Type"].unique()

array(['COP 1-1-1 DAYS SPLIT NORMAL', '3-day wrap (around pump date)', '',
       'Index', 'EFP', 'Completion of Pump over', 'Completion of Pump',
       'Other (3 Day Wrap (CmpltOP) )', 'N/A', 'Other (Custom)',
       'Fixed Dates', '3-day wrap', '3 Days - Completion of Pump', ' ',
       'Other (Day Prior to Completion of Discharge)', 'Fixed dates',
       'Float', 'Unknown', 'Fixed', 'Other (Cal Month Average (Exclude))',
       'Month average', '1/1/1', '3-Day Wrap', '3 DAY WRAP', '(Empty)',
       'Other (TBMA - Dates to be Mutually Agreed)',
       '1/1/1 (day before, day of, day after)', 'COL', 'COD',
       '1 Month - Completion of Pump', 'DEEMED DATES', '3-DAY WRAP',
       'Pump completion date'], dtype=object)

In [288]:
df['Pricing Event Type'] = df['Pricing Event Type'].astype(str)

In [289]:
df["Pricing Event Type"].unique()

array(['COP 1-1-1 DAYS SPLIT NORMAL', '3-day wrap (around pump date)', '',
       'Index', 'EFP', 'Completion of Pump over', 'Completion of Pump',
       'Other (3 Day Wrap (CmpltOP) )', 'N/A', 'Other (Custom)',
       'Fixed Dates', '3-day wrap', '3 Days - Completion of Pump', ' ',
       'Other (Day Prior to Completion of Discharge)', 'Fixed dates',
       'Float', 'Unknown', 'Fixed', 'Other (Cal Month Average (Exclude))',
       'Month average', '1/1/1', '3-Day Wrap', '3 DAY WRAP', '(Empty)',
       'Other (TBMA - Dates to be Mutually Agreed)',
       '1/1/1 (day before, day of, day after)', 'COL', 'COD',
       '1 Month - Completion of Pump', 'DEEMED DATES', '3-DAY WRAP',
       'Pump completion date'], dtype=object)

In [290]:
df['Pricing Event Type'] = df['Pricing Event Type'].map(lambda string: clean_string(string))

In [291]:
df["Pricing Event Type"].unique()

array(['COP 1-1-1 DAYS SPLIT NORMAL', '3-day wrap (around pump date)',
       None, 'Index', 'EFP', 'Completion of Pump over',
       'Completion of Pump', 'Other (3 Day Wrap (CmpltOP) )',
       'Other (Custom)', 'Fixed Dates', '3-day wrap',
       '3 Days - Completion of Pump',
       'Other (Day Prior to Completion of Discharge)', 'Fixed dates',
       'Float', 'Unknown', 'Fixed', 'Other (Cal Month Average (Exclude))',
       'Month average', '1/1/1', '3-Day Wrap', '3 DAY WRAP',
       'Other (TBMA - Dates to be Mutually Agreed)',
       '1/1/1 (day before, day of, day after)', 'COL', 'COD',
       '1 Month - Completion of Pump', 'DEEMED DATES', '3-DAY WRAP',
       'Pump completion date'], dtype=object)

#### Pricing Event Date  

In [292]:
df['Pricing Event Date'] = df["Pricing Event Date"].astype(str)

In [293]:
df["Pricing Event Date"].unique()

array(['2025-01-18 06:00:00', '2025-02-04 06:00:00',
       '2025-03-07 06:00:00', '', '3/1/2025 to 3/31/2025',
       '1/23/2025 to 1/27/2025', '1/7/2025 to 1/9/2025',
       '1/3/2025 to 1/3/2025', '2025-01-12', 'unavailable', '(Empty)',
       'N/A', '2/3/2025 to 2/28/2025', '1/6/2025 to 1/6/2025',
       '1/31/2025 to 1/31/2025', '2025-02-14',
       '2025-01-09,2025-01-10,2025-01-13', '2025-01-31', '2025-02-07',
       '2025-02-03', '1/7/2025 to 1/7/2025', '3/24/2025 to 3/26/2025',
       '01/08/2025', '2025-01-28', '1/8/2025 to 1/8/2025',
       '6/30/2025 to 6/30/2025', '5/30/2025 to 5/30/2025',
       '4/30/2025 to 4/30/2025', '2/28/2025 to 2/28/2025',
       '3/31/2025 to 3/31/2025', '1/1/1900 to 1/1/1900', '2025-03-03',
       '2025-01-19', '2025-01-16,2025-01-17,2025-01-21', '2025-02-28',
       '2025-01-09', '2025-01-07', '01/16/2025, 01/17/2025, 01/21/2025',
       '2025-01-20', '2/24/2025 to 2/24/2025',
       '01/17/2025, 01/21/2025, 01/22/2025', '01/16/2025 to 01/25/202

In [294]:
# i = 0
# for x in df['Pricing Event Date']:
#     print(str(i), x)
#     print(str_to_date(x))
#     i = i + 1

In [295]:
df['Pricing Event Date'] = df["Pricing Event Date"].map(lambda x: str_to_date(x))

In [296]:
df["Pricing Event Date"].unique()

array([datetime.date(2025, 1, 18), datetime.date(2025, 2, 4),
       datetime.date(2025, 3, 7), NaT, datetime.date(2025, 3, 31),
       datetime.date(2025, 1, 27), datetime.date(2025, 1, 9),
       datetime.date(2025, 1, 3), datetime.date(2025, 1, 12),
       datetime.date(2025, 2, 28), datetime.date(2025, 1, 6),
       datetime.date(2025, 1, 31), datetime.date(2025, 2, 14),
       datetime.date(2025, 2, 7), datetime.date(2025, 2, 3),
       datetime.date(2025, 1, 7), datetime.date(2025, 3, 26),
       datetime.date(2025, 1, 8), datetime.date(2025, 1, 28),
       datetime.date(2025, 6, 30), datetime.date(2025, 5, 30),
       datetime.date(2025, 4, 30), datetime.date(1900, 1, 1),
       datetime.date(2025, 3, 3), datetime.date(2025, 1, 19),
       datetime.date(2025, 1, 20), datetime.date(2025, 2, 24),
       datetime.date(2025, 1, 25), datetime.date(2025, 1, 13),
       datetime.date(2025, 1, 15), datetime.date(2025, 1, 22),
       datetime.date(2025, 1, 17), datetime.date(2025, 1, 14)

#### Pricing Start Date 

In [297]:
df['Pricing Start Date'] = df["Pricing Start Date"].astype(str)

In [298]:
df["Pricing Start Date"].unique()

array(['2025-01-16 06:00:00', '2025-02-03 06:00:00',
       '2025-03-06 06:00:00', '', '2025-03-01', '2025-01-23',
       '2025-01-07', '2025-01-03', '2025-01-10', '(Empty)', 'N/A',
       '2025-02-03', '2025-01-06', '2025-01-31', '2025-02-13',
       '2025-01-16', '2025-01-09', '01/07/2025', '01/15/2025',
       '01/24/2025', '01/13/2025', '01/23/2025', '01/28/2025',
       '01/16/2025', '01/08/2025', '2025-02-28', '2025-03-24',
       '2025-01-08', '2025-06-30', '2025-05-30', '2025-04-30',
       '2025-03-31', '1900-01-01', '2025-03-03', '2025-01-17',
       '2025-02-01', '2025-04-01', '2025-05-01', '2025-06-02',
       '2025-02-24', '2025-01-30', '2025-02-06', '01/21/2025',
       '01/09/2025', '2025-01-25', '2025-01-15', '2025-01-14',
       '2025-01-13', '2025-02-28 06:00:00', '2025-01-24',
       '2025-02-13 06:00:00', '2025-01-24 06:00:00',
       '2025-02-10 06:00:00', '2025-02-14 06:00:00',
       '2025-02-24 06:00:00', '2025-01-21', '2024-12-30', '2025-01-02',
       '2025-01

In [299]:
df['Pricing Start Date'] = df["Pricing Start Date"].map(lambda x: str_to_date(x))

In [300]:
df["Pricing Start Date"].unique()

array([datetime.date(2025, 1, 16), datetime.date(2025, 2, 3),
       datetime.date(2025, 3, 6), NaT, datetime.date(2025, 3, 1),
       datetime.date(2025, 1, 23), datetime.date(2025, 1, 7),
       datetime.date(2025, 1, 3), datetime.date(2025, 1, 10),
       datetime.date(2025, 1, 6), datetime.date(2025, 1, 31),
       datetime.date(2025, 2, 13), datetime.date(2025, 1, 9),
       datetime.date(2025, 1, 15), datetime.date(2025, 1, 24),
       datetime.date(2025, 1, 13), datetime.date(2025, 1, 28),
       datetime.date(2025, 1, 8), datetime.date(2025, 2, 28),
       datetime.date(2025, 3, 24), datetime.date(2025, 6, 30),
       datetime.date(2025, 5, 30), datetime.date(2025, 4, 30),
       datetime.date(2025, 3, 31), datetime.date(1900, 1, 1),
       datetime.date(2025, 3, 3), datetime.date(2025, 1, 17),
       datetime.date(2025, 2, 1), datetime.date(2025, 4, 1),
       datetime.date(2025, 5, 1), datetime.date(2025, 6, 2),
       datetime.date(2025, 2, 24), datetime.date(2025, 1, 30),
 

#### Pricing End Date 

In [301]:
df['Pricing End Date'] = df["Pricing End Date"].astype(str)

In [302]:
df["Pricing End Date"].unique()

array(['2025-01-21 06:00:00', '2025-02-05 06:00:00',
       '2025-03-10 05:00:00', '', '2025-03-31', '2025-01-27',
       '2025-01-09', '2025-01-03', '2025-01-14', '(Empty)', 'N/A',
       '2025-02-28', '2025-01-06', '2025-01-31', '2025-02-18',
       '2025-01-17', '2025-01-24', '2025-01-13', '01/07/2025',
       '01/17/2025', '01/28/2025', '01/14/2025', '01/24/2025',
       '01/29/2025', '01/10/2025', '2025-01-07', '2025-03-26',
       '01/08/2025', '2025-01-08', '2025-02-03', '2025-06-30',
       '2025-05-30', '2025-04-30', '1900-01-01', '2025-03-03',
       '2025-01-22', '2025-01-21', '2025-02-24', '2025-02-10',
       '01/21/2025', '01/09/2025', '2025-01-26', '2025-01-15',
       '2025-01-16', '01/13/2025', '2025-02-28 06:00:00', '2025-02-26',
       '2025-02-18 06:00:00', '2025-01-28 06:00:00',
       '2025-02-12 06:00:00', '2025-02-19 06:00:00',
       '2025-02-26 06:00:00', '01/15/2025', '2025-01-10', '2025-02-07',
       '01/27/2025', '2025-02-10 06:00:00', '2025-01-31 06:00:00

In [303]:
# for x in df["Pricing End Date"]:
#     print(x)
#     print(str_to_date(x))

In [304]:
df['Pricing End Date'] = df["Pricing End Date"].map(lambda x: str_to_date(x))

In [305]:
df["Pricing End Date"].unique()

array([datetime.date(2025, 1, 21), datetime.date(2025, 2, 5),
       datetime.date(2025, 3, 10), NaT, datetime.date(2025, 3, 31),
       datetime.date(2025, 1, 27), datetime.date(2025, 1, 9),
       datetime.date(2025, 1, 3), datetime.date(2025, 1, 14),
       datetime.date(2025, 2, 28), datetime.date(2025, 1, 6),
       datetime.date(2025, 1, 31), datetime.date(2025, 2, 18),
       datetime.date(2025, 1, 17), datetime.date(2025, 1, 24),
       datetime.date(2025, 1, 13), datetime.date(2025, 1, 7),
       datetime.date(2025, 1, 28), datetime.date(2025, 1, 29),
       datetime.date(2025, 1, 10), datetime.date(2025, 3, 26),
       datetime.date(2025, 1, 8), datetime.date(2025, 2, 3),
       datetime.date(2025, 6, 30), datetime.date(2025, 5, 30),
       datetime.date(2025, 4, 30), datetime.date(1900, 1, 1),
       datetime.date(2025, 3, 3), datetime.date(2025, 1, 22),
       datetime.date(2025, 2, 24), datetime.date(2025, 2, 10),
       datetime.date(2025, 1, 26), datetime.date(2025, 1, 1

#### NYMEX Price 

In [306]:
df["NYMEX Price"] = df["NYMEX Price"].astype(str)

In [307]:
df["NYMEX Price"].unique()

array(['', ' ', '205.0', '0.0', '205.37', '2.2262', '(Empty)',
       'FEB RB 1/6', 'REB RB 1/6', 'N/A', '203.55', '235.52', '236.0',
       '206.0', '2.3078', '236.63', '234.87', '2.0866', '2.0666',
       '2.3663', 'FEB RB 1/7', '201.02', '221.98', '224.47', '226.02',
       '203.87', '225.83', '235.07', '201.0', '226.0', '1.9702', '2.0387',
       '237.82', '235.8', '227.3', '202.83', '2.78037666666667',
       '2.83691', '227.0', '202.5', 'Apr rb 3/3', 'Feb rb 1/10', '210.03',
       '213.35', '210.02999999999997', '253.33', '210.46', '252.53',
       '254.0', '249.87', '248.0', '252.5', '2.4845', '2.2646', '2.4985',
       '218.61', '215.89', '207.49', '2.5549', '246.0', '261.72',
       '256.63', '215.25', '212.0', 'NOT APPLICABLE', 'NA'], dtype=object)

In [308]:
df['NYMEX Price'] = df['NYMEX Price'].map(lambda string: str_to_float(string))

In [309]:
df["NYMEX Price"].unique()

array([         nan, 205.        ,   0.        , 205.37      ,
         2.2262    , 203.55      , 235.52      , 236.        ,
       206.        ,   2.3078    , 236.63      , 234.87      ,
         2.0866    ,   2.0666    ,   2.3663    , 201.02      ,
       221.98      , 224.47      , 226.02      , 203.87      ,
       225.83      , 235.07      , 201.        , 226.        ,
         1.9702    ,   2.0387    , 237.82      , 235.8       ,
       227.3       , 202.83      ,   2.78037667,   2.83691   ,
       227.        , 202.5       , 210.03      , 213.35      ,
       210.03      , 253.33      , 210.46      , 252.53      ,
       254.        , 249.87      , 248.        , 252.5       ,
         2.4845    ,   2.2646    ,   2.4985    , 218.61      ,
       215.89      , 207.49      ,   2.5549    , 246.        ,
       261.72      , 256.63      , 215.25      , 212.        ])

In [310]:
# NYMEX
# If <100 Then *100
# 2.53 ------> 253

In [311]:
df['NYMEX Price'] = np.where(df['NYMEX Price']<100, df['NYMEX Price']*100, df['NYMEX Price'])

In [312]:
df["NYMEX Price"].unique()

array([         nan, 205.        ,   0.        , 205.37      ,
       222.62      , 203.55      , 235.52      , 236.        ,
       206.        , 230.78      , 236.63      , 234.87      ,
       208.66      , 206.66      , 201.02      , 221.98      ,
       224.47      , 226.02      , 203.87      , 225.83      ,
       235.07      , 201.        , 226.        , 197.02      ,
       237.82      , 235.8       , 227.3       , 202.83      ,
       278.03766667, 283.691     , 227.        , 202.5       ,
       210.03      , 213.35      , 210.03      , 253.33      ,
       210.46      , 252.53      , 254.        , 249.87      ,
       248.        , 252.5       , 248.45      , 226.46      ,
       249.85      , 218.61      , 215.89      , 207.49      ,
       255.49      , 246.        , 261.72      , 256.63      ,
       215.25      , 212.        ])

In [313]:
df['NYMEX Price'] = np.where(df['NYMEX Price']==0, np.nan, df['NYMEX Price'])

In [314]:
df["NYMEX Price"].unique()

array([         nan, 205.        , 205.37      , 222.62      ,
       203.55      , 235.52      , 236.        , 206.        ,
       230.78      , 236.63      , 234.87      , 208.66      ,
       206.66      , 201.02      , 221.98      , 224.47      ,
       226.02      , 203.87      , 225.83      , 235.07      ,
       201.        , 226.        , 197.02      , 237.82      ,
       235.8       , 227.3       , 202.83      , 278.03766667,
       283.691     , 227.        , 202.5       , 210.03      ,
       213.35      , 210.03      , 253.33      , 210.46      ,
       252.53      , 254.        , 249.87      , 248.        ,
       252.5       , 248.45      , 226.46      , 249.85      ,
       218.61      , 215.89      , 207.49      , 255.49      ,
       246.        , 261.72      , 256.63      , 215.25      ,
       212.        ])

In [315]:
df["NYMEX Price"] = df["NYMEX Price"].astype(str)

In [316]:
df['NYMEX Price'] = np.where(df['NYMEX Price']=='nan', '', df['NYMEX Price'])

In [317]:
df["NYMEX Price"].unique()

array(['', '205.0', '205.37', '222.62', '203.55', '235.52', '236.0',
       '206.0', '230.77999999999997', '236.63', '234.87',
       '208.65999999999997', '206.66000000000003', '201.02', '221.98',
       '224.47', '226.02', '203.87', '225.83', '235.07', '201.0', '226.0',
       '197.01999999999998', '237.82', '235.8', '227.3', '202.83',
       '278.037666666667', '283.69100000000003', '227.0', '202.5',
       '210.03', '213.35', '210.02999999999997', '253.33', '210.46',
       '252.53', '254.0', '249.87', '248.0', '252.5',
       '248.45000000000002', '226.46', '249.85', '218.61', '215.89',
       '207.49', '255.49', '246.0', '261.72', '256.63', '215.25', '212.0'],
      dtype=object)

#### Cash Price (Cents per Gallon)

In [318]:
df["Cash Price"].unique()

array(['', ' ', 216.0, 261.767, 213.87, 216.37, 2.2262, 216.87, '219.66',
       236.82, 'N/A', 231.28, 207.55, 209.8, 209.55, 241.02, 239.52,
       242.02, 240.0, 231.0, 2.3078, 2.5578, 238.63, 236.75, 242.13,
       274.64, 239.63, 238.87, 0.0, 210.05, 2.0866, 2.0666, 2.2713, 2.55,
       '1/8 FEB HO + .01', 199.02, 237.98, 240.47, 227.87, 233.83, 201.02,
       206.0, 205.0, 197.02, 238.45, 234.02, 236.69, 239.07, 235.07,
       202.0, 198.0, 201.0, 234.0, 280.0, 281.0, 1.9702, 2.2787, 234.82,
       236.32, 242.32, 236.8, 236.3, 278.69, 206.83, 198.33, 218.83,
       2.78037666666667, 2.83691, 241.82, '206', '205', 236.0,
       '1/2 Feb HO - 11', 'plus fees', '1/2 Feb HO + 0.03',
       '1/8 FEB HO - .1150', '1/10 Feb HO +.14', 2.73, 234.41, 241.47,
       269.97, 272.99, 311.0, 315.85, 'TBD', 241.35, 239.35, 245.53,
       239.56, 314.0, 217.03, 218.03, 235.53, 240.35, 213.02999999999997,
       252.33, '313', '316', 316.0, 313.0, 249.87, 253.53, 256.53, 247.45,
       248.45, 2

In [319]:
df["Cash Price"] = df["Cash Price"].map(lambda string: str_to_float(string))

In [320]:
df["Cash Price"].unique()

array([         nan, 216.        , 261.767     , 213.87      ,
       216.37      ,   2.2262    , 216.87      , 219.66      ,
       236.82      , 231.28      , 207.55      , 209.8       ,
       209.55      , 241.02      , 239.52      , 242.02      ,
       240.        , 231.        ,   2.3078    ,   2.5578    ,
       238.63      , 236.75      , 242.13      , 274.64      ,
       239.63      , 238.87      ,   0.        , 210.05      ,
         2.0866    ,   2.0666    ,   2.2713    ,   2.55      ,
       199.02      , 237.98      , 240.47      , 227.87      ,
       233.83      , 201.02      , 206.        , 205.        ,
       197.02      , 238.45      , 234.02      , 236.69      ,
       239.07      , 235.07      , 202.        , 198.        ,
       201.        , 234.        , 280.        , 281.        ,
         1.9702    ,   2.2787    , 234.82      , 236.32      ,
       242.32      , 236.8       , 236.3       , 278.69      ,
       206.83      , 198.33      , 218.83      ,   2.78

In [321]:
# if x<100 Then x*100
df['Cash Price'] = np.where(df['Cash Price']<100, df['Cash Price']*100 , df['Cash Price'])

In [322]:
df["Cash Price"].unique()

array([         nan, 216.        , 261.767     , 213.87      ,
       216.37      , 222.62      , 216.87      , 219.66      ,
       236.82      , 231.28      , 207.55      , 209.8       ,
       209.55      , 241.02      , 239.52      , 242.02      ,
       240.        , 231.        , 230.78      , 255.78      ,
       238.63      , 236.75      , 242.13      , 274.64      ,
       239.63      , 238.87      ,   0.        , 210.05      ,
       208.66      , 206.66      , 227.13      , 255.        ,
       199.02      , 237.98      , 240.47      , 227.87      ,
       233.83      , 201.02      , 206.        , 205.        ,
       197.02      , 238.45      , 234.02      , 236.69      ,
       239.07      , 235.07      , 202.        , 198.        ,
       201.        , 234.        , 280.        , 281.        ,
       197.02      , 234.82      , 236.32      , 242.32      ,
       236.8       , 236.3       , 278.69      , 206.83      ,
       198.33      , 218.83      , 278.03766667, 283.69

#### Broker Used Entity


In [323]:
df["Broker Used Entity"] = df["Broker Used Entity"].astype(str)

In [324]:
df["Broker Used Entity"].unique()

array(['Ocean Solutions En Us', 'Liquidity Us', 'None',
       'SAGE REFINED PRODUCTS, LTD.', 'n/a', '',
       'Ocean Solutions Energy LLC', 'IVG', 'OCEAN SOLUTIONS ENERGY LLC',
       'SAGE', '(Empty)', 'Ocean Solutions Energy', 'Pacifica Energy',
       'Sage Refined Products', 'Ocean Solutions Us', 'Jake Energy Us',
       'Sage Refined Products, Ltd.', 'Sage Refined Products Ltd',
       'Sage Refined Us', 'Jake Energy, LLC', 'JAKE ENERGY, LLC',
       'Ocean Solutions', 'Ocean Solutions, LLC', 'N/A', 'Slo Broker Us',
       'Sage Refined Products Ltd - Financial', 'SLO Broker - Financial',
       'SLO Broker', 'SLO', 'Jake Energy LLC', 'SLO BROKER',
       'SAGE REFINED PRODUCTS, LTD', 'DIRECT', 'OCEAN SOLUTIONS',
       'Ocean Solutions Energy LLC - Financial', 'Liquidity Energy LLC',
       'Liquidity Energy Us', 'Jake Energy LLC - Financial',
       'LIQUIDITY ENERGY', 'NOT APPLICABLE', 'Ocean Solutions LLC',
       'Jake Energy'], dtype=object)

In [325]:
# (Empty) or NA or None: ""
df["Broker Used Entity"] = df["Broker Used Entity"].map(lambda string: '' if string in ['', '(Empty)', 'n/a', 'None']  else string)

In [326]:
df["Broker Used Entity"].unique()

array(['Ocean Solutions En Us', 'Liquidity Us', '',
       'SAGE REFINED PRODUCTS, LTD.', 'Ocean Solutions Energy LLC', 'IVG',
       'OCEAN SOLUTIONS ENERGY LLC', 'SAGE', 'Ocean Solutions Energy',
       'Pacifica Energy', 'Sage Refined Products', 'Ocean Solutions Us',
       'Jake Energy Us', 'Sage Refined Products, Ltd.',
       'Sage Refined Products Ltd', 'Sage Refined Us', 'Jake Energy, LLC',
       'JAKE ENERGY, LLC', 'Ocean Solutions', 'Ocean Solutions, LLC',
       'N/A', 'Slo Broker Us', 'Sage Refined Products Ltd - Financial',
       'SLO Broker - Financial', 'SLO Broker', 'SLO', 'Jake Energy LLC',
       'SLO BROKER', 'SAGE REFINED PRODUCTS, LTD', 'DIRECT',
       'OCEAN SOLUTIONS', 'Ocean Solutions Energy LLC - Financial',
       'Liquidity Energy LLC', 'Liquidity Energy Us',
       'Jake Energy LLC - Financial', 'LIQUIDITY ENERGY',
       'NOT APPLICABLE', 'Ocean Solutions LLC', 'Jake Energy'],
      dtype=object)

#### Broker Used Entity Contact Name

In [327]:
df["Broker Used Entity Contact Name"] = df["Broker Used Entity Contact Name"].astype(str)

In [328]:
df["Broker Used Entity Contact Name"].unique()

array(['Travis Dorsey', 'Anthony Roberts', '', 'n/a', 'unavailable',
       '(Empty)', 'N/A', 'Russell Harris', 'Bianca Dumont',
       'Jason Aigner', 'Jeff Johnston', 'Russ Harris', 'Jeff Johnson',
       'Adam Morley', 'harris', 'morley', 'Marex Spectron',
       'Biance Dumont', 'BIANCA DUMONT', 'DORSEY', 'aigner',
       'Ocean Solutions Energy', 'DIRECT', 'Jake Energy', 'Dumont',
       'ANTHONY ROBERTS', 'NOT APPLICABLE'], dtype=object)

#### Reported to OPIS

In [329]:
df["Reported to OPIS"] = df["Reported to OPIS"].astype(str)

In [330]:
df["Reported to OPIS"].unique()

array(['N/A', 'No', 'None', 'Yes', 'No - Not Reported By Shell', 'Broker',
       'unavailable', 'Y', 'N', 'unknown', '', 'Not reported by Company',
       'No - Not Reported by Shell ', 'No - Not Reported by Shell', 'NO',
       ' ', 'y', 'NONE',
       'not by NGL; unkown if by broker or counterparty'], dtype=object)

### Sell to negative

In [331]:
df['Contract Volume'] = np.where(df['Seller or Buyer']=='Sell', -df['Contract Volume'].abs(), df['Contract Volume'])

In [332]:
df['Contract Volume'] = np.where(df['Seller or Buyer']=='Buy', df['Contract Volume'].abs(), df['Contract Volume'])

In [333]:
df['Seller or Buyer'].unique()

array(['Sell', 'Buy', 'In-Tank Transfer'], dtype=object)

### Clean NAs values

In [334]:
df = df[['timestamps'] + list_order_col]

In [335]:
for col in df.columns:
    print(col)
    if df[col].dtype==object:
        # df[col] = df[col].map(lambda string: None if string in ['NA', 'n/a', '', '(Empty)', 'None', 'nan'] else string)
        df[col] = df[col].map(lambda string: clean_string(string))

timestamps
Report Period
Company Name
Company ID Number
Date Filed
Date of Contract
Contract ID Number
Position Sequence Number
CPID
Trading Counterparty Entity
Trading Counterparty Entity Contact Name
Spot Market Trading Location
Seller or Buyer
Product Type
Product Name
Contract Volume
Contract Delivery Month
Delivery Method
Marine Vessel Name
Marine Vessel IMO Number
Delivery Location
Pipeline Delivery Sub-cycle
Delivery Start Date
Delivery End Date
Price Basis Method
Reference Product
Reference Month
Price Differential vs. Reference
Pricing Event Type
Pricing Event Date
Pricing Start Date
Pricing End Date
NYMEX Price
Cash Price
Broker Used Entity
Broker Used Entity Contact Name
Reported to OPIS
File form


In [336]:
df["Reported to OPIS"].unique()

array([None, 'No', 'Yes', 'No - Not Reported By Shell', 'Broker',
       'unavailable', 'Y', 'N', 'unknown', 'Not reported by Company',
       'No - Not Reported by Shell ', 'No - Not Reported by Shell', 'NO',
       'y', 'NONE', 'not by NGL; unkown if by broker or counterparty'],
      dtype=object)

In [337]:
df["Broker Used Entity Contact Name"].unique()

array(['Travis Dorsey', 'Anthony Roberts', None, 'unavailable',
       'Russell Harris', 'Bianca Dumont', 'Jason Aigner', 'Jeff Johnston',
       'Russ Harris', 'Jeff Johnson', 'Adam Morley', 'harris', 'morley',
       'Marex Spectron', 'Biance Dumont', 'BIANCA DUMONT', 'DORSEY',
       'aigner', 'Ocean Solutions Energy', 'DIRECT', 'Jake Energy',
       'Dumont', 'ANTHONY ROBERTS'], dtype=object)

## 2. Save Files

### Save to csv file

In [338]:
filename = "Trading_Cleaned.csv"

df.to_csv(filename, na_rep='')
# df.to_csv(os.path.join(path_CEC_Fernando_D354, filename), na_rep='')

df.to_csv(os.path.join(data_path, filename), na_rep='')
df.to_csv(os.path.join(path_CEC_Fernando_D354_data, filename), na_rep='')

### Save to REPORT

In [339]:
yyyymmdd = str(now.year) + "{:02d}".format(now.month) + "{:02d}".format(now.day)
yyyymmdd

'20250117'

In [340]:
df_Submitted = df_Submitted.reset_index(drop=True)
df_Submitted["N"] = df_Submitted.index
first_column = df_Submitted.pop('N') 
df_Submitted.insert(0, 'N', first_column)
df_Submitted['N'] = df_Submitted['N'] + 1
df_Submitted

,N,timestamps,Report Period,Company Name,Company ID Number,Date Filed,Date of Contract,Contract ID Number,Position Sequence Number,CPID,...,Pricing Event Type,Pricing Event Date,Pricing Start Date,Pricing End Date,NYMEX Price,Cash Price,Broker Used Entity,Broker Used Entity Contact Name,Reported to OPIS,File form
0,1,2025-01-06 06:57:22,2025-01-03,bp Products North America Inc.,BPN-0916161800,2025-01-06,2024-12-17 06:00:00,U240060424,2.0,U240060424-2,...,COP 1-1-1 DAYS SPLIT NORMAL,2025-01-18 06:00:00,2025-01-16 06:00:00,2025-01-21 06:00:00,,,Ocean Solutions En Us,Travis Dorsey,N/A,P:\ADMIN\PIIRA\CEC_PIIRA Student Assistants\En...
1,2,2025-01-06 06:57:22,2025-01-03,bp Products North America Inc.,BPN-0916161800,2025-01-06,2025-01-03 06:00:00,U250000527,1.0,U250000527-1,...,COP 1-1-1 DAYS SPLIT NORMAL,2025-02-04 06:00:00,2025-02-03 06:00:00,2025-02-05 06:00:00,,,Liquidity Us,Anthony Roberts,N/A,P:\ADMIN\PIIRA\CEC_PIIRA Student Assistants\En...
2,3,2025-01-06 06:57:22,2025-01-03,bp Products North America Inc.,BPN-0916161800,2025-01-06,2025-01-03 06:00:00,U250000530,1.0,U250000530-1,...,COP 1-1-1 DAYS SPLIT NORMAL,2025-03-07 06:00:00,2025-03-06 06:00:00,2025-03-10 05:00:00,,,Liquidity Us,Anthony Roberts,N/A,P:\ADMIN\PIIRA\CEC_PIIRA Student Assistants\En...
3,4,2025-01-06 07:20:16,1/3 - 1/5/2025,Tartan Oil LLC,,2025-01-06,2025-01-03,BPN25TP0008,1.0,,...,3-day wrap (around pump date),,,,,,None,,No,P:\ADMIN\PIIRA\CEC_PIIRA Student Assistants\En...
4,5,2025-01-06 07:20:16,1/3 - 1/5/2025,Tartan Oil LLC,,2025-01-06,2025-01-03,VAL25PE0003,1.0,,...,,,,,205.0,216.0,"SAGE REFINED PRODUCTS, LTD.",,No,P:\ADMIN\PIIRA\CEC_PIIRA Student Assistants\En...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
658,659,2025-01-17 10:01:10,2025-01-17,"Ultramar, Inc. / Valero Refining Co. - California",100100.0,2025-01-16,01/16/2025,8346351-1,unavailable,unavailable,...,,unavailable,01/17/2025,01/22/2025,,,SLO BROKER,unavailable,unavailable,P:\ADMIN\PIIRA\CEC_PIIRA Student Assistants\En...
659,660,2025-01-17 10:01:10,2025-01-17,"Ultramar, Inc. / Valero Refining Co. - California",100100.0,2025-01-16,01/16/2025,8346473-1,unavailable,unavailable,...,,unavailable,01/23/2025,01/27/2025,,,SLO BROKER,unavailable,unavailable,P:\ADMIN\PIIRA\CEC_PIIRA Student Assistants\En...
660,661,2025-01-17 10:01:10,2025-01-17,"Ultramar, Inc. / Valero Refining Co. - California",100100.0,2025-01-16,01/16/2025,8346482-1,unavailable,unavailable,...,,unavailable,,,,300,,unavailable,unavailable,P:\ADMIN\PIIRA\CEC_PIIRA Student Assistants\En...
661,662,2025-01-17 10:01:10,2025-01-17,"Ultramar, Inc. / Valero Refining Co. - California",100100.0,2025-01-16,01/16/2025,8347606-1,unavailable,unavailable,...,,unavailable,01/17/2025,01/22/2025,,,SLO BROKER,unavailable,unavailable,P:\ADMIN\PIIRA\CEC_PIIRA Student Assistants\En...


In [341]:
for col in df_Submitted.columns:
    df_Submitted[col] = df_Submitted[col].astype(str)

In [342]:
df_cleaned = df.copy()
df_cleaned = df_cleaned.reset_index(drop=True)
df_cleaned["N"] = df_cleaned.index
first_column = df_cleaned.pop('N') 
df_cleaned.insert(0, 'N', first_column)
df_cleaned['N'] = df_cleaned['N'] + 1
df_cleaned

,N,timestamps,Report Period,Company Name,Company ID Number,Date Filed,Date of Contract,Contract ID Number,Position Sequence Number,CPID,...,Pricing Event Type,Pricing Event Date,Pricing Start Date,Pricing End Date,NYMEX Price,Cash Price,Broker Used Entity,Broker Used Entity Contact Name,Reported to OPIS,File form
0,1,2025-01-06 06:57:22,2025-01-03,BP,BPN-0916161800,2025-01-06,2024-12-17,U240060424,2,U240060424-2,...,COP 1-1-1 DAYS SPLIT NORMAL,2025-01-18,2025-01-16,2025-01-21,None,NaN,Ocean Solutions En Us,Travis Dorsey,None,P:\ADMIN\PIIRA\CEC_PIIRA Student Assistants\En...
1,2,2025-01-06 06:57:22,2025-01-03,BP,BPN-0916161800,2025-01-06,2025-01-03,U250000527,1,U250000527-1,...,COP 1-1-1 DAYS SPLIT NORMAL,2025-02-04,2025-02-03,2025-02-05,None,NaN,Liquidity Us,Anthony Roberts,None,P:\ADMIN\PIIRA\CEC_PIIRA Student Assistants\En...
2,3,2025-01-06 06:57:22,2025-01-03,BP,BPN-0916161800,2025-01-06,2025-01-03,U250000530,1,U250000530-1,...,COP 1-1-1 DAYS SPLIT NORMAL,2025-03-07,2025-03-06,2025-03-10,None,NaN,Liquidity Us,Anthony Roberts,None,P:\ADMIN\PIIRA\CEC_PIIRA Student Assistants\En...
3,4,2025-01-06 07:20:16,2025-01-05,Tartan,None,2025-01-06,2025-01-03,BPN25TP0008,1,None,...,3-day wrap (around pump date),NaT,NaT,NaT,None,NaN,None,None,No,P:\ADMIN\PIIRA\CEC_PIIRA Student Assistants\En...
4,5,2025-01-06 07:20:16,2025-01-05,Tartan,None,2025-01-06,2025-01-03,VAL25PE0003,1,None,...,None,NaT,NaT,NaT,205.0,216.0,"SAGE REFINED PRODUCTS, LTD.",None,No,P:\ADMIN\PIIRA\CEC_PIIRA Student Assistants\En...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
658,659,2025-01-17 10:01:10,2025-01-17,Valero,100100.0,2025-01-16,2025-01-16,8346351-1,No Info.,No Info.,...,None,NaT,2025-01-17,2025-01-22,None,NaN,SLO BROKER,unavailable,unavailable,P:\ADMIN\PIIRA\CEC_PIIRA Student Assistants\En...
659,660,2025-01-17 10:01:10,2025-01-17,Valero,100100.0,2025-01-16,2025-01-16,8346473-1,No Info.,No Info.,...,None,NaT,2025-01-23,2025-01-27,None,NaN,SLO BROKER,unavailable,unavailable,P:\ADMIN\PIIRA\CEC_PIIRA Student Assistants\En...
660,661,2025-01-17 10:01:10,2025-01-17,Valero,100100.0,2025-01-16,2025-01-16,8346482-1,No Info.,No Info.,...,None,NaT,NaT,NaT,None,300.0,None,unavailable,unavailable,P:\ADMIN\PIIRA\CEC_PIIRA Student Assistants\En...
661,662,2025-01-17 10:01:10,2025-01-17,Valero,100100.0,2025-01-16,2025-01-16,8347606-1,No Info.,No Info.,...,None,NaT,2025-01-17,2025-01-22,None,NaN,SLO BROKER,unavailable,unavailable,P:\ADMIN\PIIRA\CEC_PIIRA Student Assistants\En...


# Validation

In [343]:
dict_Trading_validation = {}

### Marine Vessel Name

In [344]:
list_N = list(df_cleaned[np.logical_and(df_cleaned['Delivery Method']=='Marine Vessel', df_cleaned['Marine Vessel Name'].isnull())]['N'])

In [345]:
dict_Trading_validation['Marine Vessel Name'] = list_N

### Marine Vessel IMO Number

In [346]:
list_N = list(df_cleaned[np.logical_and(df_cleaned['Delivery Method']=='Marine Vessel', df_cleaned['Marine Vessel IMO Number'].isnull())]['N'])

In [347]:
dict_Trading_validation['Marine Vessel IMO Number'] = list_N

### Delivery End Date

In [348]:
list_N = list(df_cleaned[np.logical_and(df_cleaned['Pricing Event Type']=='Floating Price', 
                                        df_cleaned['Delivery End Date'].isnull())]['N'])

In [349]:
dict_Trading_validation['Delivery End Date'] = list_N

### Reference Product

In [350]:
list_N = list(df_cleaned[np.logical_and(df_cleaned['Price Basis Method'].isin(['Floating-Event', 'Fixed', 'Floating-Price']), 
                                        df_cleaned['Reference Product'].isnull())]['N'])

In [351]:
dict_Trading_validation['Reference Product'] = list_N

### Reference Month

In [352]:
list_N = list(df_cleaned[np.logical_and(df_cleaned['Price Basis Method'].isin(['Fixed']), 
                                        df_cleaned['Reference Month'].isnull())]['N'])

In [353]:
dict_Trading_validation['Reference Month'] = list_N

### Pricing Event Type

In [354]:
list_N = list(df_cleaned[np.logical_and(df_cleaned['Price Basis Method'].isin(['EFP', 'Fixed']), 
                                        df_cleaned['Pricing Event Type'].isnull())]['N'])

In [355]:
dict_Trading_validation['Pricing Event Type'] = list_N

### Cash Price

In [356]:
list_N = list(df_cleaned[np.logical_and(df_cleaned['Price Basis Method'].isin(['Floating-Price']), 
                                        df_cleaned['Cash Price'].isnull())]['N'])

In [357]:
dict_Trading_validation['Cash Price'] = list_N

## Create df_validation

In [358]:
list_var_validations = [ 'Marine Vessel Name', 'Marine Vessel IMO Number', 'Delivery End Date', 'Reference Product',
    'Reference Month', 'Pricing Event Type', 'Cash Price']

In [359]:
df_validation = pd.DataFrame()
df_validation['N'] = df_cleaned['N'].copy()

for var in list_var_validations:
    df_validation[var] = None
    df_validation[var] = np.where(df_validation['N'].isin(dict_Trading_validation[var]), 'FLAG', df_validation[var])

# Save

In [360]:
filename = "Trading_Validation.csv"

df_validation.to_csv(filename, na_rep='')
# df_validation.to_csv(os.path.join(path_CEC_Fernando_D354, filename), na_rep='')

df_validation.to_csv(os.path.join(data_path, filename), na_rep='')
df_validation.to_csv(os.path.join(path_CEC_Fernando_D354_data, filename), na_rep='')

In [361]:
wb = xw.Book('D354 Clean and Validation Report.xlsx')

# Cleaned
ws = wb.sheets['Cleaned']
ws.clear_contents()
ws["A1"].options(pd.DataFrame, header=1, index=False, expand='table').value = df_cleaned

# Submitted
ws = wb.sheets['Submitted']    
ws.clear_contents()
ws["A1"].options(pd.DataFrame, header=1, index=False, expand='table').value = df_Submitted

# Validation
ws = wb.sheets['Validation']
ws.clear_contents()
ws["A1"].options(pd.DataFrame, header=1, index=False, expand='table').value = df_validation

# Files
ws = wb.sheets['Files']
ws.clear_contents()
ws["A1"].options(pd.DataFrame, header=1, index=False, expand='table').value = df_files

wb.save()
wb.close()

In [362]:
shutil.copy('D354 Clean and Validation Report.xlsx', os.path.join(path_CEC_Fernando_D354, 'D354 Clean and Validation Report.xlsx'))

'P:\\ADMIN\\PIIRA\\CEC_Fernando\\D354\\D354 Clean and Validation Report.xlsx'

In [363]:
print('report date: '+report_date[:4]+'-'+report_date[4:6]+'-'+report_date[6:9])
print('report time: '+str(now.hour).zfill(2)+":"+str(now.minute).zfill(2))
end_time = time.time()
elapsed_time = end_time - start_time
print(f"Total execution time {str(math.trunc(elapsed_time/60)).zfill(2)}:{str(round(elapsed_time-math.trunc(elapsed_time/60)*60)).zfill(2)}")

report date: 2025-01-17
report time: 11:32
Total execution time 01:01
